In [2]:
import random, copy
import os
from collections import defaultdict

class JadwalKuliah:
    def __init__(
            self, 
            kode_matkul, kode_dosen, sks_akademik, kode_ruangan, kapasitas, 
            hari, jam_mulai, jam_selesai, 
            tipe_kelas, program_studi):
        self.kode_matkul = kode_matkul
        self.kode_dosen = kode_dosen
        self.sks_akademik = sks_akademik
        self.kode_ruangan = kode_ruangan
        self.kapasitas = kapasitas
        self.hari = hari
        self.jam_mulai = jam_mulai
        self.jam_selesai = jam_selesai
        self.tipe_kelas = tipe_kelas  # 'TEORI' atau 'PRAKTIKUM'
        self.program_studi = program_studi

    def __repr__(self):
        return f"<{self.kode_matkul}, {self.kode_dosen}, {self.sks_akademik}, {self.kode_ruangan}, {self.hari}, {self.jam_mulai}, {self.jam_selesai}, {self.tipe_kelas}>\n"

# TO BE CHECKED:
# (15)  Jadwal Ruangan Bertabrakan                                  >> ruangan_bentrok           (DONE)
# (15)  Jadwal Dosen Bertabrakan                                    >> dosen_bentrok             (DONE)
# (15)  Jadwal Dosen dan Asisten Berjalan Bersamaan                 >> asdos_nabrak_dosen        (DONE)
# (15)  Kelas Dosen atau Asisten Hilang atau Tidak Lengkap          >> kelas_gaib                (DONE)
# (15)  Solo Team Teaching                                          >> solo_team                 (DONE)
# (10)  Beban SKS Dosen melebihi 12 sks                             >> dosen_overdosis           (DONE)
# (10)  Matkul berlangsung sebelum pukul 7 atau sesudah pukul 19    >> diluar_jam_kerja          (DONE)
# (10)  Cek Total Kelas Bisa Cangkup Semua Mahasiswa                >> kapasitas_kelas_terbatas  (DONE)
# (5)   Tidak Sesuai dengan permintaan / request dosen              >> melanggar_preferensi      (DONE)
BOBOT_PENALTI = {
    "ruangan_bentrok": 15,
    "dosen_bentrok": 15,
    "asdos_nabrak_dosen": 15,
    "kelas_gaib": 15,
    "solo_team": 15,
    "dosen_overdosis": 10,
    "diluar_jam_kerja": 10,
    "kapasitas_kelas_terbatas": 10,
    "melanggar_preferensi": 5,

    "weekend_class": 10,
    "istirahat": 5,
    "salah_tipe_ruangan": 3,
}

matakuliah_list = [{'kode': 'TC531', 'nama': 'TEORI BAHASA DAN AUTOMATA', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'TEORI', 'dosen_ajar': ['MAGDALENA A. INEKE PAKERENG, M.KOM', 'PROF. DR. IR. EKO SEDIYONO, M.KOM.', 'ERYKA PANDU EKALIADEWI, S.T., M.ENG', 'YESSICA NATALIANI, PH.D', 'HENDRY, M.KOM., PH.D.', 'BUDHI KRISTIANTO, PH.D'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300}, {'kode': 'TC532', 'nama': 'STATISTIKA DAN PROBABILITAS', 'sks_akademik': 2, 'sks_bayar': 2, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'TEORI', 'dosen_ajar': ['ALZ DANNY WOWOR, S.SI., M.CS.', 'DANIEL KURNIAWAN, M.PD.', 'NATALIA DYAH BAYU WARSANTI, S.SI', 'EDNA MARIA, S.KOM, M.PD', 'RICHARD MAYOPU, PH.D', 'RADIUS TANONE, S.KOM., M.CS., PH.D'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300}, {'kode': 'TC533', 'nama': 'JARINGAN KOMPUTER', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'asistensi': True, 'tipe_kelas': 'TEORI', 'tipe_kelas_asistensi': 'TEORI', 'dosen_ajar': ['DR. INDRASTANTI R. WIDIASARI, M.T.', 'DIAN WIDIYANTO CHANDRA, S.KOM., M.CS.', 'THEOPHILUS HERMAN WELLEM, M.S., PH.D', 'RIKO YUDISTIRA, S.KOM', 'ERWIEN CHRISTIANTO, S.KOM, M.CS', 'DR. IRWAN SEMBIRING, ST., M.KOM.'], 'kelompok': None, 'jumlah_mahasiswa': 300}, {'kode': 'TC534', 'nama': 'ALGORITMA DAN STRUKTUR DATA', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'asistensi': True, 'tipe_kelas': 'PRAKTIKUM', 'tipe_kelas_asistensi': 'PRAKTIKUM', 'dosen_ajar': ['DR. SUPRIHADI, S.SI., M.KOM.', 'VIKKY APRELIA WINDARNI, S.KOM., M.CS', 'DR. ADI NUGROHO, S.T., MMSI.', 'YOS RICHARD BEEH, S.T., M.CS.', 'CHERYL LOUISA LOEDWYCA HISKIA, S.SI'], 'kelompok': None, 'jumlah_mahasiswa': 300}, {'kode': 'TC535', 'nama': 'INTERAKSI MANUSIA KOMPUTER', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'asistensi': True, 'tipe_kelas': 'TEORI', 'tipe_kelas_asistensi': 'TEORI', 'dosen_ajar': ['YEREMIA ALFA SUSETYO, S.KOM., M.CS', 'NINA SETIYAWATI, S.KOM., M.CS.', 'VYORBIGGER B. OPPIER, S.KOM', 'DANANG WIDIATMOKO, S.DS', 'CHERYL LOUISA LOEDWYCA HISKIA, S.SI', 'SUHARYADI, S.KOM., M.CS.'], 'kelompok': None, 'jumlah_mahasiswa': 300}, {'kode': 'TC631', 'nama': 'SISTEM OPERASI', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'asistensi': True, 'tipe_kelas': 'TEORI', 'tipe_kelas_asistensi': 'TEORI', 'dosen_ajar': ['ARIYA DWIKA CAHYONO, S. KOM, M.T.', 'VIKKY APRELIA WINDARNI, S.KOM., M.CS', 'SRI WINARSO MARTYAS EDI, S.KOM, M.CS', 'ERWIEN CHRISTIANTO, S.KOM, M.CS', 'CHERYL LOUISA LOEDWYCA HISKIA, S.SI', 'DANANG WIDIATMOKO, S.DS', 'RIKO YUDISTIRA, S.KOM'], 'kelompok': None, 'jumlah_mahasiswa': 300}, {'kode': 'TC632', 'nama': 'REKAYASA PERANGKAT LUNAK', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'asistensi': True, 'tipe_kelas': 'PRAKTIKUM', 'tipe_kelas_asistensi': 'PRAKTIKUM', 'dosen_ajar': ['DR. SRI YULIANTO J. P., S.SI., M.KOM.', 'VYORBIGGER B. OPPIER, S.KOM', 'SUHARYADI, S.KOM., M.CS.', 'DR. ADI NUGROHO, S.T., MMSI.', 'ARIYA DWIKA CAHYONO, S. KOM, M.T.', 'SRI WINARSO MARTYAS EDI, S.KOM, M.CS', 'VIKKY APRELIA WINDARNI, S.KOM., M.CS', 'FELIX DAVID, S.KOM., M.CS.'], 'kelompok': None, 'jumlah_mahasiswa': 300}, {'kode': 'TC633', 'nama': 'METODE PENELITIAN', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'TEORI', 'dosen_ajar': ['DR. INDRASTANTI R. WIDIASARI, M.T.', 'BUDHI KRISTIANTO, PH.D', 'MAGDALENA A. INEKE PAKERENG, M.KOM', 'PROF. DR. KRISTOKO DWI HARTOMO, M.KOM.', 'DR. SUPRIHADI, S.SI., M.KOM.', 'RADIUS TANONE, S.KOM., M.CS., PH.D', 'DR. CHARITAS FIBRIANI, S.KOM, M.ENG', 'RICHARD MAYOPU, PH.D'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300}, {'kode': 'TC634', 'nama': 'KEAMANAN DATA', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'TEORI', 'dosen_ajar': ['MAGDALENA A. INEKE PAKERENG, M.KOM', 'ALZ DANNY WOWOR, S.SI., M.CS.', 'THEOPHILUS HERMAN WELLEM, M.S., PH.D', 'ANGLING KUSUMANDHITA RADEN, S.T., M.M.', 'DR. SUPRIHADI, S.SI., M.KOM.', 'FAJAR AYUNINGTYAS, S.KOM', 'RIKO YUDISTIRA, S.KOM'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300}, {'kode': 'TC635', 'nama': 'MACHINE LEARNING', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['PROF. HINDRIYANTO DWI PURNOMO, PH.D.', 'DR. CHARITAS FIBRIANI, S.KOM, M.ENG', 'HENDRY, M.KOM., PH.D.', 'CHRISTINE DEWI, PH.D.', 'ARIYA DWIKA CAHYONO, S. KOM, M.T.', 'NINA SETIYAWATI, S.KOM., M.CS.', 'DR. TEGUH WAHYONO, S.KOM, M.CS'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300}, {'kode': 'TC712', 'nama': 'PEMROGRAMAN MOBILE', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['PRATYAKSA OCSA NUGRAHA SAIAN, S.KOM., M.T', 'YOS RICHARD BEEH, S.T., M.CS.'], 'kelompok': None, 'asistensi': None, 'jumlah_kelas': 2, 'jumlah_mahasiswa': 300}, {'kode': 'TC713', 'nama': 'UI/UX', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['NINA SETIYAWATI, S.KOM., M.CS.'], 'kelompok': None, 'asistensi': None, 'jumlah_kelas': 1, 'jumlah_mahasiswa': 300}, {'kode': 'TC716', 'nama': 'WEB SERVICES', 'kelompok': 'BIDANG MINAT SOFTWARE ENGINEERING', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'bidang': ['SOFTWARE'], 'asistensi': None, 'jumlah_kelas': 1, 'jumlah_mahasiswa': 300}, {'kode': 'TC719', 'nama': 'PEMROGRAMAN BERORIENTASI PLATFORM', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['YEREMIA ALFA SUSETYO, S.KOM., M.CS'], 'kelompok': None, 'asistensi': None, 'jumlah_kelas': 1, 'jumlah_mahasiswa': 300}, {'kode': 'TC720', 'nama': 'GRAFIKA KOMPUTER', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['PRATYAKSA OCSA NUGRAHA SAIAN, S.KOM., M.T'], 'kelompok': None, 'asistensi': None, 'jumlah_kelas': 1, 'jumlah_mahasiswa': 300}, {'kode': 'TC731', 'nama': 'METODOLOGI DATA SCIENCE', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['PROF. HINDRIYANTO DWI PURNOMO, PH.D.', 'HENDRY, M.KOM., PH.D.'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 2}, {'kode': 'TC734', 'nama': 'PEMODELAN SPASIAL', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['DR. SRI YULIANTO J. P., S.SI., M.KOM.'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 1}, {'kode': 'TC737', 'nama': 'PEMROSESAN BAHASA ALAMI', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['EVANGS MAILOA, S.KOM., M.CS.'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 2}, {'kode': 'TC754', 'nama': 'ROUTING DAN SWITCHING 1', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'bidang': ['CISCO'], 'asistensi': True, 'dosen_ajar': ['DR. WIWIN SULISTYO, S.T., M.KOM.', 'DR. INDRASTANTI R. WIDIASARI, M.T.'], 'tipe_kelas_asistensi': 'PRAKTIKUM', 'kelompok': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 2}, {'kode': 'TC757', 'nama': 'KEAMANAN JARINGAN', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['DIAN WIDIYANTO CHANDRA, S.KOM., M.CS.'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 1}, {'kode': 'TC760', 'nama': 'ROUTING DAN SWITCHING 2', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'bidang': ['CISCO'], 'asistensi': True, 'dosen_ajar': ['DIAN WIDIYANTO CHANDRA, S.KOM., M.CS.'], 'tipe_kelas_asistensi': 'PRAKTIKUM', 'kelompok': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 1}, {'kode': 'TC762', 'nama': 'KEAMANAN SIBER', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['DR. WIWIN SULISTYO, S.T., M.KOM.'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 1}, {'kode': 'TC781', 'nama': 'SERTIFIKASI SOFTWARE ENGINEERING - ORACLE', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['DR. ADI NUGROHO, S.T., MMSI.', 'SRI WINARSO MARTYAS EDI, S.KOM, M.CS', 'DR. SUPRIHADI, S.SI., M.KOM.', 'PRATYAKSA OCSA NUGRAHA SAIAN, S.KOM., M.T', 'SUHARYADI, S.KOM., M.CS.', 'NINA SETIYAWATI, S.KOM., M.CS.', 'DR. SRI YULIANTO J. P., S.SI., M.KOM.', 'FELIX DAVID, S.KOM., M.CS.'], 'jumlah_dosen': 2, 'team_teaching': True, 'kelompok': None, 'asistensi': None, 'jumlah_kelas': 4, 'jumlah_mahasiswa': 300}, {'kode': 'TC782', 'nama': 'SERTIFIKASI DATA SCIENCE - ORACLE', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['YOS RICHARD BEEH, S.T., M.CS.', 'EVANGS MAILOA, S.KOM., M.CS.', 'ARIYA DWIKA CAHYONO, S. KOM, M.T.', 'RADIUS TANONE, S.KOM., M.CS., PH.D'], 'jumlah_dosen': 2, 'team_teaching': True, 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 2}, {'kode': 'TC783', 'nama': 'SERTIFIKASI NETWORK ENGINEERING - RSNA', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'bidang': ['CISCO'], 'dosen_ajar': ['DIAN WIDIYANTO CHANDRA, S.KOM., M.CS.', 'DR. IRWAN SEMBIRING, ST., M.KOM.', 'DR. INDRASTANTI R. WIDIASARI, M.T.', 'DR. WIWIN SULISTYO, S.T., M.KOM.'], 'jumlah_dosen': 2, 'team_teaching': True, 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 2}, {'kode': 'TC784', 'nama': 'VEHICULAR COMMUNICATION', 'sks_akademik': 3, 'sks_bayar': 4, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'TEORI', 'dosen_ajar': ['TEGUH INDRA BAYU, PH.D'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 1}, {'kode': 'TC785', 'nama': 'ARSITEKTUR DAN DESAIN SISTEM  JARINGAN', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'TEORI', 'dosen_ajar': ['THEOPHILUS HERMAN WELLEM, M.S., PH.D'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 1}, {'kode': 'TC786', 'nama': 'COMPUTER VISION WITH APPLICATION', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['RADIUS TANONE, S.KOM., M.CS., PH.D'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 1}, {'kode': 'TC787', 'nama': 'COMPUTATIONAL JOURNALISM', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['RICHARD MAYOPU, PH.D'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 1}, {'kode': 'TC788', 'nama': 'BIO-INFORMATIC', 'sks_akademik': 3, 'sks_bayar': 3, 'prodi': 'S1 TEKNIK INFORMATIKA', 'tipe_kelas': 'PRAKTIKUM', 'dosen_ajar': ['BUDHI KRISTIANTO, PH.D'], 'kelompok': None, 'asistensi': None, 'jumlah_mahasiswa': 300, 'jumlah_kelas': 1}]

dosen_list = [{
  "nip": "67564",
  "nama": "ALZ DANNY WOWOR, S.SI., M.CS.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "DATA"
  ],
  "matkul_ajar": [
    "STATISTIKA DAN PROBABILITAS",
    "KEAMANAN DATA"
  ]
},
{
  "nip": "67909",
  "nama": "CHRISTINE DEWI, PH.D.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "DATA"
  ],
  "matkul_ajar": [
    "MACHINE LEARNING"
  ]
},
{
  "nip": "67530",
  "nama": "DIAN WIDIYANTO CHANDRA, S.KOM., M.CS.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "NETWORK"
  ],
  "matkul_ajar": [
    "JARINGAN KOMPUTER",
    "SERTIFIKASI NETWORK ENGINEERING - RSNA",
    "KEAMANAN JARINGAN",
    "ROUTING DAN SWITCHING 2"
  ]
},
{
  "nip": "67003",
  "nama": "PROF. DR. IR. EKO SEDIYONO, M.KOM.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "TEORI BAHASA DAN AUTOMATA"
  ]
},
{
  "nip": "67920",
  "nama": "EVANGS MAILOA, S.KOM., M.CS.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "DATA"
  ],
  "matkul_ajar": [
    "SERTIFIKASI DATA SCIENCE - ORACLE",
    "PEMROSESAN BAHASA ALAMI"
  ]
},
{
  "nip": "67555",
  "nama": "HENDRY, M.KOM., PH.D.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "TEORI BAHASA DAN AUTOMATA",
    "MACHINE LEARNING",
    "METODOLOGI DATA SCIENCE"
  ],
  "preferensi": {
      "hindari_hari": ["SENIN", "KAMIS"]
  }
},
{
  "nip": "67513",
  "nama": "PROF. HINDRIYANTO DWI PURNOMO, PH.D.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "MACHINE LEARNING",
    "METODOLOGI DATA SCIENCE"
  ]
},
{
  "nip": "67523",
  "nama": "DR. INDRASTANTI R. WIDIASARI, M.T.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "NETWORK"
  ],
  "matkul_ajar": [
    "JARINGAN KOMPUTER",
    "METODE PENELITIAN",
    "SERTIFIKASI NETWORK ENGINEERING - RSNA",
    "ROUTING DAN SWITCHING 1"
  ]
},
{
  "nip": "67515",
  "nama": "DR. ADI NUGROHO, S.T., MMSI.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "DATA"
  ],
  "matkul_ajar": [
    "ALGORITMA DAN STRUKTUR DATA",
    "REKAYASA PERANGKAT LUNAK",
    "SERTIFIKASI SOFTWARE ENGINEERING - ORACLE"
  ]
},
{
  "nip": "67002",
  "nama": "MAGDALENA A. INEKE PAKERENG, M.KOM",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "DATA"
  ],
  "matkul_ajar": [
    "TEORI BAHASA DAN AUTOMATA",
    "METODE PENELITIAN",
    "KEAMANAN DATA"
  ],
  "preferensi": {
      "hindari_hari": ["SENIN", "JUMAT"]
  }
},
{
  "nip": "67938",
  "nama": "NINA SETIYAWATI, S.KOM., M.CS.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "INTERAKSI MANUSIA KOMPUTER",
    "MACHINE LEARNING",
    "SERTIFIKASI SOFTWARE ENGINEERING - ORACLE",
    "UI/UX"
  ]
},
{
  "nip": "68975",
  "nama": "PRATYAKSA OCSA NUGRAHA SAIAN, S.KOM., M.T",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "SERTIFIKASI SOFTWARE ENGINEERING - ORACLE",
    "PEMROGRAMAN MOBILE",
    "GRAFIKA KOMPUTER"
  ]
},
{
  "nip": "67565",
  "nama": "RADIUS TANONE, S.KOM., M.CS., PH.D",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "STATISTIKA DAN PROBABILITAS",
    "METODE PENELITIAN",
    "SERTIFIKASI DATA SCIENCE - ORACLE",
    "COMPUTER VISION WITH APPLICATION"
  ]
},
{
  "nip": "67506",
  "nama": "DR. SRI YULIANTO J. P., S.SI., M.KOM.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "REKAYASA PERANGKAT LUNAK",
    "SERTIFIKASI SOFTWARE ENGINEERING - ORACLE",
    "PEMODELAN SPASIAL"
  ]
},
{
  "nip": "67545",
  "nama": "DR. SUPRIHADI, S.SI., M.KOM.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "matkul_ajar": [
    "ALGORITMA DAN STRUKTUR DATA",
    "METODE PENELITIAN",
    "KEAMANAN DATA",
    "SERTIFIKASI SOFTWARE ENGINEERING - ORACLE"
  ]
},
{
  "nip": "67554",
  "nama": "TEGUH INDRA BAYU, PH.D",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "NETWORK"
  ],
  "matkul_ajar": [
    "VEHICULAR COMMUNICATION"
  ]
},
{
  "nip": "67501",
  "nama": "THEOPHILUS HERMAN WELLEM, M.S., PH.D",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "NETWORK"
  ],
  "matkul_ajar": [
    "JARINGAN KOMPUTER",
    "KEAMANAN DATA",
    "ARSITEKTUR DAN DESAIN SISTEM  JARINGAN"
  ]
},
{
  "nip": "67507",
  "nama": "DR. WIWIN SULISTYO, S.T., M.KOM.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "NETWORK"
  ],
  "matkul_ajar": [
    "SERTIFIKASI NETWORK ENGINEERING - RSNA",
    "KEAMANAN SIBER",
    "ROUTING DAN SWITCHING 1"
  ]
},
{
  "nip": "67977",
  "nama": "YEREMIA ALFA SUSETYO, S.KOM., M.CS",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "INTERAKSI MANUSIA KOMPUTER",
    "PEMROGRAMAN BERORIENTASI PLATFORM"
  ]
},
{
  "nip": "67532",
  "nama": "YOS RICHARD BEEH, S.T., M.CS.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "ALGORITMA DAN STRUKTUR DATA",
    "PEMROGRAMAN MOBILE",
    "SERTIFIKASI DATA SCIENCE - ORACLE"
  ]
},
{
  "nip": "56907",
  "nama": "BUDHI KRISTIANTO, PH.D",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "NETWORK"
  ],
  "matkul_ajar": [
    "TEORI BAHASA DAN AUTOMATA",
    "METODE PENELITIAN",
    "BIO-INFORMATIC"
  ]
},
{
  "nip": "67566",
  "nama": "SUHARYADI, S.KOM., M.CS.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "INTERAKSI MANUSIA KOMPUTER",
    "REKAYASA PERANGKAT LUNAK",
    "SERTIFIKASI SOFTWARE ENGINEERING - ORACLE"
  ]
},
{
  "nip": "55905",
  "nama": "ARIYA DWIKA CAHYONO, S. KOM, M.T.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "DATA"
  ],
  "matkul_ajar": [
    "SISTEM OPERASI",
    "REKAYASA PERANGKAT LUNAK",
    "MACHINE LEARNING",
    "SERTIFIKASI DATA SCIENCE - ORACLE"
  ]
},
{
  "nip": "67941",
  "nama": "RICHARD MAYOPU, PH.D",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "pakar": [
    "DATA"
  ],
  "matkul_ajar": [
    "STATISTIKA DAN PROBABILITAS",
    "METODE PENELITIAN",
    "COMPUTATIONAL JOURNALISM"
  ]
},
{
  "nip": "67578",
  "nama": "FAJAR AYUNINGTYAS, S.KOM",
  "status": "TIDAK_TETAP",
  "pakar": [
    "DATA",
    "NETWORK",
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "KEAMANAN DATA"
  ]
},
{
  "nip": "67570",
  "nama": "RIKO YUDISTIRA, S.KOM",
  "status": "TIDAK_TETAP",
  "pakar": [
    "NETWORK"
  ],
  "matkul_ajar": [
    "JARINGAN KOMPUTER",
    "SISTEM OPERASI",
    "KEAMANAN DATA"
  ]
},
{
  "nip": "67922",
  "nama": "CHERYL LOUISA LOEDWYCA HISKIA, S.SI",
  "status": "TIDAK_TETAP",
  "pakar": [
    "SOFTWARE",
    "NETWORK"
  ],
  "matkul_ajar": [
    "ALGORITMA DAN STRUKTUR DATA",
    "INTERAKSI MANUSIA KOMPUTER",
    "SISTEM OPERASI"
  ]
},
{
  "nip": "67810",
  "nama": "DANANG WIDIATMOKO, S.DS",
  "status": "TIDAK_TETAP",
  "pakar": [
    "SOFTWARE",
    "DATA"
  ],
  "matkul_ajar": [
    "INTERAKSI MANUSIA KOMPUTER",
    "SISTEM OPERASI"
  ]
},
{
  "nip": "67916",
  "nama": "EDNA MARIA, S.KOM, M.PD",
  "status": "TIDAK_TETAP",
  "pakar": [
    "NETWORK"
  ],
  "matkul_ajar": [
    "STATISTIKA DAN PROBABILITAS"
  ]
},
{
  "nip": "67927",
  "nama": "HILDA SARANITA MOMONGAN, M.PD",
  "status": "TIDAK_TETAP",
  "pakar": [
    "BAHASA"
  ],
  "matkul_ajar": [
    "BAHASA INGGRIS"
  ]
},
{
  "nip": "67574",
  "nama": "MARTZA MERRY SWASTIKASARI, S.KOM, M.KOM",
  "status": "TIDAK_TETAP",
  "pakar": [
    "NETWORK"
  ]
},
{
  "nip": "68972",
  "nama": "VIKKY APRELIA WINDARNI, S.KOM., M.CS",
  "status": "TIDAK_TETAP",
  "pakar": [
    "NETWORK",
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "ALGORITMA DAN STRUKTUR DATA",
    "SISTEM OPERASI",
    "REKAYASA PERANGKAT LUNAK"
  ]
},
{
  "nip": "67947",
  "nama": "VYORBIGGER B. OPPIER, S.KOM",
  "status": "TIDAK_TETAP",
  "pakar": [
    "SOFTWARE"
  ],
  "matkul_ajar": [
    "PENGUJIAN SISTEM",
    "INTERAKSI MANUSIA KOMPUTER",
    "REKAYASA PERANGKAT LUNAK"
  ]
},
{
  "nip": "67975",
  "nama": "ERYKA PANDU EKALIADEWI, S.T., M.ENG",
  "status": "TIDAK_TETAP",
  "pakar": [
    "MATEMATIKA"
  ],
  "matkul_ajar": [
    "TEORI BAHASA DAN AUTOMATA"
  ]
},
{
  "nip": "68973",
  "nama": "NATALIA DYAH BAYU WARSANTI, S.SI",
  "status": "TIDAK_TETAP",
  "pakar": [
    "MATEMATIKA",
    "DATA"
  ],
  "matkul_ajar": [
    "STATISTIKA DAN PROBABILITAS"
  ]
},
{
  "nip": "67967",
  "nama": "ANGLING KUSUMANDHITA RADEN, S.T., M.M.",
  "status": "TIDAK_TETAP",
  "matkul_ajar": [
    "KEAMANAN DATA"
  ]
},
{
  "nip": "11955",
  "nama": "DANIEL KURNIAWAN, M.PD.",
  "status": "TIDAK_TETAP",
  "matkul_ajar": [
    "STATISTIKA DAN PROBABILITAS"
  ]
},
{
  "nip": "00000",
  "nama": "KEN TESTING 1",
  "status": "TETAP",
  "prodi": "D3 TEKNIK INFORMATIKA"
},
{
  "nip": "00001",
  "nama": "KEN TESTING 2",
  "status": "TETAP",
  "prodi": "S1 SISTEM INFORMASI",
  "pakar": [
    "TESTER"
  ]
},
{
  "nip": "67925",
  "nama": "HANITA YULIA, M.PD.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "matkul_ajar": [
    "BAHASA INGGRIS"
  ]
},
{
  "nip": "67524",
  "nama": "YESSICA NATALIANI, PH.D",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "matkul_ajar": [
    "TEORI BAHASA DAN AUTOMATA"
  ]
},
{
  "nip": "67512",
  "nama": "DR. IRWAN SEMBIRING, ST., M.KOM.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "matkul_ajar": [
    "JARINGAN KOMPUTER",
    "SERTIFIKASI NETWORK ENGINEERING - RSNA"
  ]
},
{
  "nip": "67955",
  "nama": "ERWIEN CHRISTIANTO, S.KOM, M.CS",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "matkul_ajar": [
    "JARINGAN KOMPUTER",
    "SISTEM OPERASI"
  ]
},
{
  "nip": "67005",
  "nama": "FELIX DAVID, S.KOM., M.CS.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "matkul_ajar": [
    "REKAYASA PERANGKAT LUNAK",
    "SERTIFIKASI SOFTWARE ENGINEERING - ORACLE"
  ]
},
{
  "nip": "67001",
  "nama": "PROF. IR. DANIEL H. F. MANONGGA, M.SC., PH.D.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA"
},
{
  "nip": "67517",
  "nama": "DR. DRA. ADE IRIANI, MM.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA"
},
{
  "nip": "67004",
  "nama": "PROF. DR. KRISTOKO DWI HARTOMO, M.KOM.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "matkul_ajar": [
    "METODE PENELITIAN"
  ]
},
{
  "nip": "67508",
  "nama": "T. ARIE SETIAWAN PRASIDA, S.T., M.CS.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA"
},
{
  "nip": "55909",
  "nama": "SRI WINARSO MARTYAS EDI, S.KOM, M.CS",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "matkul_ajar": [
    "SISTEM OPERASI",
    "REKAYASA PERANGKAT LUNAK",
    "SERTIFIKASI SOFTWARE ENGINEERING - ORACLE"
  ]
},
{
  "nip": "67503",
  "nama": "DR. TEGUH WAHYONO, S.KOM, M.CS",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "matkul_ajar": [
    "MACHINE LEARNING"
  ]
},
{
  "nip": "67942",
  "nama": "RISSAL EFENDI, M.KOM.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA"
},
{
  "nip": "67560",
  "nama": "DR. CHARITAS FIBRIANI, S.KOM, M.ENG",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA",
  "matkul_ajar": [
    "METODE PENELITIAN",
    "MACHINE LEARNING"
  ]
},
{
  "nip": "67511",
  "nama": "KRISMIYATI, S.PD., M.A., PH.D.",
  "status": "TETAP",
  "prodi": "S1 TEKNIK INFORMATIKA"
}]

ruang_list = [
  { "kode": "FTI300", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },
  { "kode": "FTI301", "tipe_ruangan": "TEORI", "kapasitas": 19, "plot": ["S1 PERPUSTAKAAN DAN SAINS INFORMASI"] },
  { "kode": "FTI302", "tipe_ruangan": "TEORI", "kapasitas": 21, "plot": ["S1 HUBUNGAN MASYARAKAT"] },
  { "kode": "FTI306", "tipe_ruangan": "TEORI", "kapasitas": 43, "plot": ["GENERAL"] },
  { "kode": "FTI308", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },
  { "kode": "FTI329", "tipe_ruangan": "TEORI", "kapasitas": 44, "plot": ["GENERAL"] },
  { "kode": "FTI331", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },
  { "kode": "FTI333", "tipe_ruangan": "TEORI", "kapasitas": 56, "plot": ["GENERAL"] },
  { "kode": "FTI401", "tipe_ruangan": "TEORI", "kapasitas": 25, "plot": ["S2 MAGISTER SISTEM INFORMASI"] },
  { "kode": "FTI420", "tipe_ruangan": "TEORI", "kapasitas": 39, "plot": ["GENERAL"] },
  { "kode": "FTI422", "tipe_ruangan": "TEORI", "kapasitas": 45, "plot": ["GENERAL"] },
  { "kode": "FTI424", "tipe_ruangan": "TEORI", "kapasitas": 44, "plot": ["GENERAL"] },
  { "kode": "FTI504", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["S1 PENDIDIKAN TEKNIK INFORMATIKA DAN KOMPUTER"] },
  { "kode": "FTI505", "tipe_ruangan": "TEORI", "kapasitas": 21, "plot": ["GENERAL"] },
  { "kode": "FTI508", "tipe_ruangan": "TEORI", "kapasitas": 33, "plot": ["GENERAL"] },
  { "kode": "FTI511", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },

  { "kode": "FTI310", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 32, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI402", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 18, "plot": ["S1 SIA"], "specs": [] },
  { "kode": "FTI429", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 3, "plot": ["S1 DESAIN KOMUNIKASI VISUAL"], "specs": [] },
  { "kode": "FTI431", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 20, "plot": ["NETWORK", "CISCO"], "specs": [] },
  { "kode": "FTI451", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 27, "plot": ["SINARMAS"], "specs": [] },
  { "kode": "FTI453", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 7, "plot": ["D3 TEKNIK INFORMATIKA", "HARDWARE"], "specs": [] },
  { "kode": "FTI454", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 20, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI455", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 41, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI457", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI459", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI461", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["D3 TEKNIK INFORMATIKA"], "specs": [] },
  { "kode": "FTI463", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI464", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI465", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI467", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI469", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI506", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 30, "plot": ["ORACLE"], "specs": [] },
  { "kode": "FTI507", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 20, "plot": ["NETWORK", "CISCO"], "specs": [] },
  { "kode": "FTI400", "tipe_ruangan": "RAPAT", "kapasitas": 103, "plot": ["GENERAL"] }
]


In [3]:
from IPython.display import FileLink
import xlsxwriter

def export_jadwal_to_excel(jadwal_list, filename="jadwal_kuliah.xlsx"):
    fixed_headers = [
        'kode_matkul', 'nama_matkul',
        'kode_dosen', 'nama_dosen', 'sks_akademik', 
        'kode_ruangan', 'kapasitas', 
        'hari', 'jam_mulai', 'jam_selesai',
        'tipe_kelas'
    ]

    # 🔹 Group berdasarkan program_studi
    grouped = defaultdict(list)
    for jadwal in jadwal_list:
        grouped[jadwal.program_studi].append(jadwal)

    # 🔹 Buat workbook
    workbook = xlsxwriter.Workbook(filename)

    # Format header
    format_header = workbook.add_format({
        'align': 'center',
        'bg_color': '#99CCFF',
        'valign': 'vcenter',
        'bold': True
    })
    format_as = workbook.add_format({
        'align': 'center',
        'bg_color': '#b9ebc6',
    })
    format_error = workbook.add_format({
        'bg_color': '#ff0000'
    })

    # Loop setiap group dan tulis ke sheet terpisah
    for program_studi in sorted(grouped.keys()):
        sheet_name = program_studi[:31]  # Sheet name max 31 chars
        worksheet = workbook.add_worksheet(sheet_name)

        row_idx = 0
        col_widths = [len(h) for h in fixed_headers]

        # Tulis header kolom
        for col_idx, header in enumerate(fixed_headers):
            worksheet.write(row_idx, col_idx, header.replace('_', ' '), format_header)
        row_idx += 1

        # Sort dan tulis data
        # sorted_group = sorted(grouped[program_studi], key=lambda x: (x.kode_ruangan, x.hari, x.jam_mulai))
        sorted_group = sorted(grouped[program_studi], key=lambda x: x.kode_matkul)
        for jadwal in sorted_group:
            for col_idx, attr in enumerate(fixed_headers):
                if attr == "nama_matkul":
                    value = getattr(jadwal, 'kode_matkul')
                    value = next((m['nama'] for m in matakuliah_list if m['kode'] == value[:-1] or m['kode'] == value[:-4]), None)
                elif attr == "nama_dosen":
                    value = getattr(jadwal, 'kode_dosen')
                    value = next((d['nama'] for d in dosen_list if d['nip'] == value), None)
                elif attr == "kapasitas":
                    value = getattr(jadwal, "kode_matkul")
                    if value[-2:] == "AS":
                        kapasitas_dosen = next((sesi.kapasitas for sesi in jadwal_list if sesi.kode_matkul == value[:-3]), None)
                        value = kapasitas_dosen
                    else:
                        value = getattr(jadwal, attr)
                else:
                    value = getattr(jadwal, attr)

                if (attr == "jam_mulai" and int(value) < 7) or (attr == "jam_selesai" and int(value) > 19):
                    worksheet.write(row_idx, col_idx, value, format_error)
                else:
                    worksheet.write(row_idx, col_idx, value, format_as if getattr(jadwal, 'kode_dosen') == "AS" else None)

                val_len = len(str(value))
                if val_len > col_widths[col_idx]:
                    col_widths[col_idx] = val_len
            row_idx += 1

        # Set lebar kolom
        for col_idx, width in enumerate(col_widths):
            worksheet.set_column(col_idx, col_idx, width + 2)

        # Data beban dosen
        beban_dosen = {}
        for sesi in sorted_group:
            if sesi.kode_dosen != "AS":
                if sesi.kode_dosen not in beban_dosen: beban_dosen[sesi.kode_dosen] = 0
                beban_dosen[sesi.kode_dosen] += sesi.sks_akademik
        # Tulis data beban dosen
        start_col = len(fixed_headers) + 2
        row_idx = 0
        worksheet.write(row_idx, start_col, "NIP", format_header)
        worksheet.write(row_idx, start_col + 1, "Beban SKS", format_header)
        row_idx += 1
        
        for nip, sks in dict(sorted(beban_dosen.items())).items():
            worksheet.write(row_idx, start_col, nip)
            worksheet.write(row_idx, start_col + 1, sks)
            row_idx += 1

    # Simpan workbook
    workbook.close()
    print(f"✅ File '{filename}' berhasil dibuat.")
    print('Excel disimpan di:', os.getcwd())
    return FileLink(filename)

def export_ruangan_to_excel(jadwal_list, filename="jadwal_ruangan.xlsx"):
    hari = ['SENIN', 'SELASA', 'RABU', 'KAMIS', 'JUMAT', 'SABTU']
    mapping_hari = {h: i for i, h in enumerate(hari, start=2)}
    jam = list(range(7, 19+1))
    jam.append(0)
    tampilan_jam = [f"{j:02d}:00" for j in jam]
    mapping_jam = {j: i for i, j in enumerate(jam, start=2)}

    workbook = xlsxwriter.Workbook(filename)

    # 🔹 Group berdasarkan program_studi
    grouped = defaultdict(list)
    for jadwal in jadwal_list:
        grouped[jadwal.kode_ruangan].append(jadwal)

    # Format header
    format_header = workbook.add_format({
        'align': 'center',
        'bg_color': '#99CCFF',
        'valign': 'vcenter',
        'bold': True
    })

    # Loop setiap group dan tulis ke sheet terpisah
    for ruangan in sorted(grouped.keys()):
        sheet_name = ruangan[:31]  # Sheet name max 31 chars
        worksheet = workbook.add_worksheet(sheet_name)
        worksheet.set_default_row(30)
        worksheet.hide_gridlines()

        row_idx = 0

        # Tulis header kolom
        for col_idx, value in enumerate(hari):
            worksheet.write(0, col_idx + 1, value, format_header)
            worksheet.set_column(col_idx + 1, col_idx + 1, 15) # set lebar column
        for row_idx in range(len(tampilan_jam) - 1):  # hindari akses indeks terakhir + 1
            waktu_mulai = tampilan_jam[row_idx]
            waktu_selesai = tampilan_jam[row_idx + 1]
            worksheet.write(row_idx + 1, 0, f"{waktu_mulai}-{waktu_selesai}", format_header)
            worksheet.set_column(0, 0, 15) # set lebar column
        row_idx += 1

        # Sort dan tulis data
        sorted_group = sorted(grouped[ruangan], key=lambda x: (x.tipe_kelas, x.kode_ruangan, x.hari, x.jam_mulai))
        for jadwal in sorted_group:
            format_data = workbook.add_format({
                'align': 'center',
                'valign': 'vcenter',
                'text_wrap': True,
                'bg_color': random.choice(['#fce9b3', '#b5fcb3', '#fadcb9', '#b9e3fa', '#b9e3fa', '#fab9df', '#b9fae6']),
            })

            col = chr(64 + mapping_hari[jadwal.hari])
            start_row = str(mapping_jam[jadwal.jam_mulai])
            end_row = str(mapping_jam[jadwal.jam_selesai - 1])

            kode_matkul = jadwal.kode_matkul[:-1] if jadwal.kode_dosen != "AS" else jadwal.kode_matkul[:-4]
            index = jadwal.kode_matkul[-1:] if jadwal.kode_dosen != "AS" else jadwal.kode_matkul[-4:-3]
            nama_matkul = next((m['nama'] for m in matakuliah_list if m['kode'] == kode_matkul), '')
            nama_dosen = next((d['nama'] for d in dosen_list if d['nip'] == jadwal.kode_dosen), '') if jadwal.kode_dosen != "AS" else "ASISTEN"
            
            full = f"{nama_matkul} {index} - {nama_dosen}".split(' ')
            full = [value.capitalize() for value in full]
            value = ''
            for text in full:
                value += (" " + text)
            try:
                worksheet.merge_range(
                    col + start_row + ":" + col + end_row, 
                    value, 
                    format_data
                )
            except Exception as e:
                None

    # Simpan workbook
    workbook.close()
    print(f"✅ File '{filename}' berhasil dibuat.")
    print('Excel disimpan di:', os.getcwd())
    return FileLink(filename)

def tampilkan_jadwal(jadwal):
    print(f"\n📅 Jadwal Mata Kuliah Terbaik ({len(jadwal)} kelas):\n")
    print(f"{'QTY':<5}{'MK':<57}{'Dosen':<20}{'Ruang':<8}{'SKS':<10}{'Hari':<10}{'Jam Mulai':<12}{'Jam Selesai':<12}{'Tipe':<10}{'Program Studi'}")
    print("-" * 60)
    # # urutkan berdasarkan kode_matkul, hari dan jam_mulai
    # for sesi in sorted(jadwal, key=lambda x: (x.kode_matkul, x.hari, x.jam_mulai)):
    #     print(f"{sesi.kode_matkul:<8}{sesi.kode_dosen:<15}{sesi.kode_ruangan:<8}{sesi.sks_akademik:<10}{sesi.hari:<10}{sesi.jam_mulai:<12}{sesi.jam_selesai:<12}{sesi.tipe_kelas}")

    # urutkan berdasarkan kode_kelas (biar keliatan jelas yang bentrok kelas mana)
    # for sesi in sorted(jadwal, key=lambda x: (x.kode_dosen, x.hari, x.jam_mulai)):
    for sesi in sorted(jadwal, key=lambda x: (x.kode_ruangan, x.hari, x.jam_mulai, x.kode_matkul)):
    # for sesi in sorted(jadwal, key=lambda x: (x.hari, x.jam_mulai)):
    # for sesi in sorted(jadwal, key=lambda x: (x.kode_matkul, x.hari, x.jam_mulai)):
        # print(sesi.kode_matkul, sesi.kode_matkul[:-2], sesi.kode_matkul[:-5])
        nama_matkul = next((m['nama'] for m in matakuliah_list if m['kode'] == sesi.kode_matkul[:-1] or m['kode'] == sesi.kode_matkul[:-4]), '')
        nama_dosen = next((d['nama'] for d in dosen_list if d['nip'] == sesi.kode_dosen), sesi.kode_dosen) if sesi.kode_dosen != "AS" else "ASISTEN"
        print(f"{sesi.kapasitas:<5}{sesi.kode_matkul:<12}{nama_matkul:<45}{sesi.kode_dosen:<6}{'nama_dosen':<14}{sesi.kode_ruangan:<8}{sesi.sks_akademik:<10}{sesi.hari:<10}{sesi.jam_mulai:<12}{sesi.jam_selesai:<12}{sesi.tipe_kelas:<10}{sesi.program_studi}")
 

In [4]:
def convertOutputToDict(jadwal_list):
    """
    Menkonversi object jadwal menjadi dictionary.

    Returns:
        dict: Jadwal dalam bentuk dictionary.
    """

    jadwal = []

    for sesi in jadwal_list:
        s = {}
        s['kode_matkul'] = sesi.kode_matkul
        s['kode_dosen'] = sesi.kode_dosen
        s['sks_akademik'] = sesi.sks_akademik
        s['kode_ruangan'] = sesi.kode_ruangan
        s['kapasitas'] = sesi.kapasitas
        s['hari'] = sesi.hari
        s['jam_mulai'] = sesi.jam_mulai
        s['jam_selesai'] = sesi.jam_selesai
        s['tipe_kelas'] = sesi.tipe_kelas  # 'TEORI' atau 'PRAKTIKUM'
        s['program_studi'] = sesi.program_studi
        jadwal.append(s)

    return jadwal

def find_available_schedule(jadwal:list, kode:str, hari:str):
    """
    Mencari waktu yang tersedia dari jadwal yang sudah ada berdasarkan hari yang dipilih.

    Args:
        jadwal (list): List jadwal ruangan / dosen yang sudah ada.
        kode (str): Kode yang akan diperiksa (kode ruangan / nip).
        hari (str): Hari yang akan diperiksa jadwalnya.

    Returns:
        list: List jadwal yang bisa digunakan (jam).
    """
    
    pilihan_jam = list(range(7, 19 + 1)) if hari != "SABTU" else list(range(7, 13))
    
    jadwal_sesuai_hari = [sesi for sesi in jadwal[kode] if sesi['hari'] == hari] if jadwal[kode] else []
    used_jam = []
    for sesi in jadwal_sesuai_hari:
        used_jam.extend(range(sesi['jam_mulai'], sesi['jam_selesai'] + 1))

    return [h for h in pilihan_jam if h not in used_jam]

def angka_ke_huruf(n: int):
    """
    Generate alphabet (A-Z) dari angka (n) yang diberikan.

    Args:
        n (int): Angka yang ingin diparsing ke alphabet.

    Returns:
        chr|None: Alphabet yang sesuai dengan angka yang diberikan.
    """

    if 1 <= int(n) <= 26:
        return chr(64 + n)  # Karena ord('A') = 65
    else:
        return None  # Diluar jangkauan 1-26

def find_missing_course(jadwal, matakuliah_list):
    kode_matkul = []
    for matkul in matakuliah_list:
        kode_matkul.append(matkul['kode'])

    sesi_matkul = []
    for sesi in jadwal:
        if sesi.kode_dosen != "AS" and sesi.kode_matkul[:-1] not in sesi_matkul:
            sesi_matkul.append(sesi.kode_matkul[:-1])

    missing = []
    for kode in kode_matkul:
        if kode not in sesi_matkul:
            missing.append(kode)

    return missing

def sync_team_teaching(sesi_dosen, jadwal, jadwal_dosen, jadwal_ruangan):
    for sesi_lain in jadwal:
        if sesi_lain.kode_matkul == sesi_dosen.kode_matkul and sesi_lain.kode_dosen != sesi_dosen.kode_dosen:
            old_hari = sesi_lain.hari
            old_jam_mulai = sesi_lain.jam_mulai
            old_jam_selesai = sesi_lain.jam_selesai
            old_kode_ruangan = sesi_lain.kode_ruangan
            
            sesi_lain.hari = sesi_dosen.hari
            sesi_lain.jam_mulai = sesi_dosen.jam_mulai
            sesi_lain.jam_selesai = sesi_dosen.jam_selesai
            sesi_lain.kode_ruangan = sesi_dosen.kode_ruangan
            sesi_lain.kapasitas = sesi_dosen.kapasitas
            sesi_lain.tipe_kelas = sesi_dosen.tipe_kelas

            if sesi_lain.kode_dosen not in jadwal_dosen:
                jadwal_dosen[sesi_lain.kode_dosen] = []
                
            jadwal_dosen[sesi_lain.kode_dosen] = [
                detail for detail in jadwal_dosen[sesi_lain.kode_dosen]
                if not (detail['hari'] == old_hari and detail['jam_mulai'] == old_jam_mulai and detail['jam_selesai'] == old_jam_selesai)
            ]
            jadwal_dosen[sesi_lain.kode_dosen].append({
                'hari': sesi_lain.hari,
                'jam_mulai': sesi_lain.jam_mulai,
                'jam_selesai': sesi_lain.jam_selesai,
            })

            if sesi_dosen.kode_ruangan not in jadwal_ruangan:
                jadwal_ruangan[sesi_dosen.kode_ruangan] = []
            if old_kode_ruangan not in jadwal_ruangan:
                jadwal_ruangan[old_kode_ruangan] = []
                
            jadwal_ruangan[old_kode_ruangan] = [
                detail for detail in jadwal_ruangan[old_kode_ruangan]
                if not (detail['hari'] == old_hari and detail['jam_mulai'] == old_jam_mulai and detail['jam_selesai'] == old_jam_selesai)
            ]
            jadwal_ruangan[sesi_dosen.kode_ruangan].append({
                'hari': sesi_lain.hari,
                'jam_mulai': sesi_lain.jam_mulai,
                'jam_selesai': sesi_lain.jam_selesai,
            })
            
    return jadwal_dosen, jadwal_ruangan


In [5]:
def rand_dosen_pakar(list_dosen_pakar: list, dict_beban_sks_dosen: dict = {}, excluded_dosen: list = []):
    if len(list_dosen_pakar) > 1:
        list_beban_sks_dosen = [
            value for key, value in dict_beban_sks_dosen.items() 
            if key in [
                dosen['nip'] for dosen in list_dosen_pakar
                if dosen['nip'] not in excluded_dosen
            ]
        ]
        beban_sks_tertinggi_saat_ini = max(list_beban_sks_dosen)

        kandidat_dosen = [
            dosen for dosen in list_dosen_pakar
            if (
                dict_beban_sks_dosen[dosen['nip']] < beban_sks_tertinggi_saat_ini 
                if any(sks_dosen < beban_sks_tertinggi_saat_ini for sks_dosen in list_beban_sks_dosen) 
                else dict_beban_sks_dosen[dosen['nip']] <= beban_sks_tertinggi_saat_ini 
            )
        ]
        # sorted(dosen_list, key=lambda dosen: (len(dosen.get("matkul_ajar", [])), len(dosen.get('pakar', []))), reverse=True)
        
        return random.choice([ dosen for dosen in kandidat_dosen if dosen['nip'] not in excluded_dosen ] or kandidat_dosen)
    elif len(list_dosen_pakar) == 1:
        return list_dosen_pakar[0]

def rand_ruangan(list_ruangan: list, data_matkul: dict, excluded_room: list = [], forAsisten: bool = False, kapasitas_ruangan_dosen: int = 0):
    toChecked = [data_matkul["prodi"], "GENERAL"]
    toChecked.extend(data_matkul.get("bidang", []))
    ruangan_prodi = [
        ruangan for ruangan in list_ruangan
        if any(plot in ruangan["plot"] for plot in toChecked) and
            ruangan["kode"] not in excluded_room
    ]
    
    if not forAsisten:
        kandidat_ruangan = [
            ruangan for ruangan in ruangan_prodi 
            if ruangan['tipe_ruangan'] == data_matkul['tipe_kelas']
            # if ruangan["tipe_ruangan"] in ([data_matkul["tipe_kelas"]] if data_matkul.get("asistensi", None) else [data_matkul["tipe_kelas"], "RAPAT"])
        ]

        if data_matkul.get("asistensi", None):
            if data_matkul.get("tipe_kelas_asistensi", "PRAKTIKUM") == "PRAKTIKUM":
                kandidat_ruangan = [
                    ruangan for ruangan in kandidat_ruangan 
                    if ruangan["kapasitas"] < max(
                        [ 
                            ruangan["kapasitas"] for ruangan in list_ruangan 
                            if ruangan["tipe_ruangan"] == "PRAKTIKUM" 
                        ]
                    )
                ]
    elif forAsisten:
        kandidat_ruangan = [
            ruangan for ruangan in ruangan_prodi 
            if ruangan["tipe_ruangan"] == data_matkul.get("tipe_kelas_asistensi", "TEORI") and
                ruangan["kapasitas"] >= kapasitas_ruangan_dosen
        ]

    # max_room_capacity = max([ ruangan["kapasitas"] for ruangan in kandidat_ruangan ])
    
    if len(kandidat_ruangan) > 1:
        bobot_kandidat_ruangan = [
            (len(set(ruangan.get("plot", [])) & set(data_matkul.get("bidang", [])))*10 or 1) * 
                (10 if data_matkul["prodi"] in ruangan.get("plot", []) else 1) * 
                (ruangan["kapasitas"] / 10)
            for ruangan in kandidat_ruangan
        ]

        ruangan_terpilih = random.choices(
            population=kandidat_ruangan, 
            weights=bobot_kandidat_ruangan, 
            k=1)[0]
        return ruangan_terpilih
    elif len(kandidat_ruangan) == 1:
        return kandidat_ruangan[0]

In [6]:
def repair_jadwal(jadwal, matakuliah_list, dosen_list, ruang_list):
    pilihan_hari_dosen = ["SENIN", "SELASA", "RABU", "KAMIS", "JUMAT"]
    pilihan_hari_asisten = copy.deepcopy(pilihan_hari_dosen)
    pilihan_hari_asisten.append("SABTU")

    max_attempt = 10

    jadwal_dosen = {} # Cek Jadwal Dosen {'kode': [{'hari': '', 'jam_mulai': 0, 'jam_selesai': 0}]}
    jadwal_ruangan = {} # Cek Jadwal Ruangan {'kode': [{'hari': '', 'jam_mulai': 0, 'jam_selesai': 0}]}
    beban_dosen = {} # Cek SKS Dosen {'kode': 0}

    # AMBIL INITIAL
    for ruang in ruang_list:
        if ruang['kode'] not in jadwal_ruangan: jadwal_ruangan[ruang['kode']] = []
    for dosen in dosen_list:
        if dosen['nip'] not in jadwal_dosen: jadwal_dosen[dosen['nip']] = []
        if dosen['nip'] not in beban_dosen: beban_dosen[dosen['nip']] = 0

    # HAPUS DATA MATKUL DUPLIKAT
    seen_kode_matkul = set()
    filtered_jadwal = []

    for sesi in jadwal:
        isTeamTeaching = next((matkul.get('team_teaching') for matkul in matakuliah_list if matkul['kode'] == (sesi.kode_matkul[:-1] if sesi.kode_dosen != "AS" else sesi.kode_matkul[:-4])), None)
        if sesi.kode_matkul in seen_kode_matkul:
            if isTeamTeaching and sesi.kode_dosen != "AS":
                filtered_jadwal.append(sesi)
        else:
            seen_kode_matkul.add(sesi.kode_matkul)
            filtered_jadwal.append(sesi)

    jadwal = filtered_jadwal

    # LOOP PERTAMA:
    # - REPAIR DOSEN DENGAN SKS BERLEBIH (DONE)
    # - REPAIR JADWAL DOSEN BENTROK (DONE)
    # - PENUHI KAPASITAS KELAS KALAU BERKURANG
    for sesi_dosen in jadwal:
        if sesi_dosen.kode_dosen != "AS":
            matkul = next((m for m in matakuliah_list if m['kode'] == sesi_dosen.kode_matkul[:-1]), None)
            dosen = next((d for d in dosen_list if d['nip'] == sesi_dosen.kode_dosen), None)
            preferensi_hari_dosen = [d for d in pilihan_hari_dosen if d not in dosen['preferensi']['hindari_hari']] if dosen.get('preferensi') and dosen['preferensi'].get('hindari_hari') else pilihan_hari_dosen

            if matkul:
                old_hari = sesi_dosen.hari
                old_jam_mulai = sesi_dosen.jam_mulai
                old_jam_selesai = sesi_dosen.jam_selesai
                old_kode_ruangan = sesi_dosen.kode_ruangan
                
                # Repair Dosen dg SKS Berlebih
                if beban_dosen[dosen['nip']] > (12 - sesi_dosen.sks_akademik):
                    dosen_pakar = [
                        d for d in dosen_list 
                        if (d.get("prodi") == matkul['prodi'] or d['status'] == "TIDAK_TETAP") and 
                            ((
                                (d.get('nama') or '') in (matkul.get('dosen_ajar') or []) or 
                                len(set(d.get('pakar') or []) & set(matkul.get('bidang') or [])) > 0
                            ) if matkul.get('bidang') or matkul.get('dosen_ajar') 
                            else True) and 
                            d["nip"] not in [sesi_lain.kode_dosen for sesi_lain in jadwal if sesi_lain.kode_matkul == sesi_dosen.kode_matkul]
                    ]
                    if dosen_pakar:
                        sesi_dosen.kode_dosen = rand_dosen_pakar(list_dosen_pakar=dosen_pakar, dict_beban_sks_dosen=beban_dosen)["nip"]
                    
                conflict = False

                for sesi_lain in jadwal_ruangan[sesi_dosen.kode_ruangan]:
                    if sesi_dosen.hari == sesi_lain['hari']:
                        if sesi_dosen.jam_mulai < sesi_lain['jam_selesai'] and sesi_dosen.jam_selesai > sesi_lain['jam_mulai']:
                            conflict = True
                            break

                for sesi_lain in jadwal_dosen[sesi_dosen.kode_dosen]:
                    if sesi_dosen.hari == sesi_lain['hari']:
                        if sesi_dosen.jam_mulai < sesi_lain['jam_selesai'] and sesi_dosen.jam_selesai > sesi_lain['jam_mulai']:
                            conflict = True
                            break

                attempt = 1
                excluded_day = []
                excluded_room = []
                # Repair Jadwal Dosen Bentrok
                while conflict and attempt <= max_attempt:
                    # print(sesi_dosen.kode_matkul, sesi_dosen.kode_dosen, sesi_dosen.sks_akademik, sesi_dosen.hari)
                    available_room_schedule = find_available_schedule(jadwal_ruangan, sesi_dosen.kode_ruangan, sesi_dosen.hari)
                    available_lecturer_schedule = find_available_schedule(jadwal_dosen, sesi_dosen.kode_dosen, sesi_dosen.hari)

                    # Sesuaikan dengan jadwal team jika team teaching
                    if matkul.get('team_teaching'):
                        # print(f"bfr   {sesi_dosen.kode_matkul} {matkul['kode']} : {available_lecturer_schedule}")
                        for sesi_lain in jadwal:
                            if sesi_lain.kode_matkul == sesi_dosen.kode_matkul and sesi_lain.kode_dosen != sesi_dosen.kode_dosen:
                                other_lecturer_schedule = find_available_schedule(jadwal_dosen, sesi_lain.kode_dosen, sesi_dosen.hari)
                                available_lecturer_schedule = list(set(available_lecturer_schedule) & set(other_lecturer_schedule))
                        # print(f"aftr  {sesi_dosen.kode_matkul} {matkul['kode']} : {available_lecturer_schedule}")
                    
                    available_schedule = list(set(available_room_schedule) & set(available_lecturer_schedule))
                    
                    status = False
                    for jam in available_schedule:
                        rentang_waktu = list(range(jam, jam + matkul['sks_akademik'] + 1))
                        if all(r in available_schedule for r in rentang_waktu) and jam != 12:
                            status = True
                            conflict = False
                            sesi_dosen.jam_mulai = jam
                            sesi_dosen.jam_selesai = jam + matkul['sks_akademik']
                            break
                        else:
                            status = False

                    if not status:
                        excluded_day.append(sesi_dosen.hari)
                        # print('DOSEN pil ', pilihan_hari_dosen)
                        # print('DOSEN exc ', excluded_day)
                        # print('DOSEN     ', [d for d in pilihan_hari_dosen if d not in excluded_day])
                        # print('      ', available_schedule)
                        if all(d in excluded_day for d in preferensi_hari_dosen):
                            excluded_day = []
                            excluded_room.append(sesi_dosen.kode_ruangan)
                            ruang_pengganti = rand_ruangan(list_ruangan=ruang_list, data_matkul=matkul, excluded_room=excluded_room)

                            sesi_dosen.kode_ruangan = ruang_pengganti['kode']
                            sesi_dosen.kapasitas = ruang_pengganti['kapasitas']
                            sesi_dosen.tipe_kelas = ruang_pengganti['tipe_ruangan']
                        else:
                            sesi_dosen.hari = random.choice([d for d in preferensi_hari_dosen if d not in excluded_day])

                    attempt += 1
                
                beban_dosen[dosen['nip']] += sesi_dosen.sks_akademik
                if matkul.get("team_teaching"):
                    jadwal_dosen, jadwal_ruangan = sync_team_teaching(sesi_dosen=sesi_dosen, jadwal=jadwal, jadwal_dosen=jadwal_dosen, jadwal_ruangan=jadwal_ruangan)
                else:
                    jadwal_dosen[sesi_dosen.kode_dosen] = [
                        detail for detail in jadwal_dosen[sesi_dosen.kode_dosen]
                        if not (detail['hari'] == old_hari and detail['jam_mulai'] == old_jam_mulai and detail['jam_selesai'] == old_jam_selesai)
                    ]
                    jadwal_dosen[sesi_dosen.kode_dosen].append({
                        'hari': sesi_dosen.hari,
                        'jam_mulai': sesi_dosen.jam_mulai,
                        'jam_selesai': sesi_dosen.jam_selesai,
                    })
                        
                    jadwal_ruangan[old_kode_ruangan] = [
                        detail for detail in jadwal_ruangan[old_kode_ruangan]
                        if not (detail['hari'] == old_hari and detail['jam_mulai'] == old_jam_mulai and detail['jam_selesai'] == old_jam_selesai)
                    ]
                    jadwal_ruangan[sesi_dosen.kode_ruangan].append({
                        'hari': sesi_dosen.hari,
                        'jam_mulai': sesi_dosen.jam_mulai,
                        'jam_selesai': sesi_dosen.jam_selesai,
                    })

                # # NEW BACKUP (NOT DONE)
                # sukses = False
                # outer_attempt = 1
                # while not sukses and outer_attempt <= max_attempt:
                #     # CEK BENTROK DOSEN
                #     conflict = True
                #     attempt = 1
                #     excluded_dosen = []
                #     while conflict and attempt <= max_attempt:
                #         for sesi_lain in jadwal_dosen[sesi_dosen.kode_dosen]:
                #             if sesi_dosen.hari == sesi_lain['hari']:
                #                 if sesi_dosen.jam_mulai < sesi_lain['jam_selesai'] and sesi_dosen.jam_selesai > sesi_lain['jam_mulai']:
                #                     conflict = True
                #                     excluded_dosen.append(sesi.kode_dosen)

                #                     dosen_pakar = [
                #                         dosen for dosen in dosen_list
                #                         if (dosen.get("prodi") == matkul['prodi'] or dosen['status'] == "TIDAK_TETAP") and 
                #                             ((
                #                                 (dosen.get('nama') or '') in (matkul.get('dosen_ajar') or []) or 
                #                                 len(set(dosen.get('pakar') or []) & set(matkul.get('bidang') or [])) > 0
                #                             ) if matkul.get('bidang') or matkul.get('dosen_ajar') 
                #                             else True) and dosen["nip"] not in excluded_dosen
                #                     ]
                                    
                #                     if dosen_pakar:
                #                         dosen_pengganti = rand_dosen_pakar(
                #                             list_dosen_pakar=dosen_pakar, 
                #                             dict_beban_sks_dosen=beban_dosen, 
                #                             excluded_dosen=excluded_dosen
                #                         )
                #                         sesi_dosen.kode_dosen = dosen_pengganti["nip"]
                #                         attempt = 0
                #                     else: attempt = max_attempt
                #                     break
                #                 else: conflict = False
                #             else: conflict = False
                        
                #         if not conflict: break
                #         attempt += 1
                        
                #     # CEK BENTROK RUANGAN
                #     conflict = True
                #     attempt = 1
                #     excluded_room = []
                #     while conflict and attempt <= max_attempt:
                #         for sesi_lain in jadwal_ruangan[sesi_dosen.kode_ruangan]:
                #             if sesi_dosen.hari == sesi_lain['hari']:
                #                 if sesi_dosen.jam_mulai < sesi_lain['jam_selesai'] and sesi_dosen.jam_selesai > sesi_lain['jam_mulai']:
                #                     conflict = True
                #                     excluded_room.append(sesi.kode_ruangan)

                #                     ruang_pengganti = rand_ruangan(
                #                         list_ruangan=ruang_list,
                #                         data_matkul=matkul,
                #                         excluded_room=excluded_room
                #                     )
                #                     if ruang_pengganti["kode"] not in excluded_room:
                #                         sesi.kode_ruangan = ruang_pengganti["kode"]
                #                         sesi_dosen.kapasitas = ruang_pengganti['kapasitas']
                #                         sesi_dosen.tipe_kelas = ruang_pengganti['tipe_ruangan']
                #                         attempt = 0
                #                     else:
                #                         # GANTI HARI - SESUAIKAN DENGAN ALL PREFERENSI DOSEN
                #                         print("GANTI HARI")
                #                     break
                #                 else: conflict = False
                #             else: conflict = False
                        
                #         if not conflict: 
                #             for team_schedule in jadwal:
                #                 if team_schedule.kode_matkul == sesi.kode_matkul:
                #                     team_schedule.kode_ruangan = sesi.kode_ruangan
                #             break
                #         attempt += 1

                #     outer_attempt += 1
                #     # if sukses:
                #     #     beban_dosen[dosen['nip']] += sesi_dosen.sks_akademik
                #     #     jadwal_ruangan[sesi_dosen.kode_ruangan].append({'hari': sesi_dosen.hari, 'jam_mulai': sesi_dosen.jam_mulai, 'jam_selesai': sesi_dosen.jam_selesai})
                #     #     jadwal_dosen[sesi_dosen.kode_dosen].append({'hari': sesi_dosen.hari, 'jam_mulai': sesi_dosen.jam_mulai, 'jam_selesai': sesi_dosen.jam_selesai})
                #     continue

    # LOOP KEDUA:
    # 2. REPAIR JADWAL ASISTEN KALAU BENTRO K DENGAN KELAS LAIN (DONE)
    for sesi_asisten in jadwal:
        if sesi_asisten.kode_dosen == "AS": # KODE ASISTEN
            matkul = next((m for m in matakuliah_list if m['kode'] == sesi_asisten.kode_matkul[:-4]), None)

            if matkul:
                conflict = False

                sesi_dosen = next((sesi_dosen for sesi_dosen in jadwal if sesi_dosen.kode_matkul == sesi_asisten.kode_matkul[:-3]), None)
                suggested_hari_asisten = pilihan_hari_asisten[pilihan_hari_dosen.index(sesi_dosen.hari):]
                for sesi_lain in jadwal_ruangan[sesi_asisten.kode_ruangan]:
                    if sesi_asisten.hari == sesi_lain['hari']:
                        if sesi_asisten.jam_mulai < sesi_lain['jam_selesai'] and sesi_asisten.jam_selesai > sesi_lain['jam_mulai']:
                            conflict = True
                            break

                attempt = 1
                excluded_day = []
                excluded_room = []
                # 2. Repair Jadwal Asisten
                while conflict and attempt <= max_attempt:
                    # print(sesi_asisten.kode_matkul, sesi_asisten.kode_dosen, sesi_asisten.sks_akademik, sesi_asisten.hari)
                    available_room_schedule = find_available_schedule(jadwal_ruangan, sesi_asisten.kode_ruangan, sesi_asisten.hari)

                    status = False
                    for jam in available_room_schedule:
                        rentang_waktu = list(range(jam, jam + matkul['sks_akademik'] + 1))
                        if all(r in available_room_schedule for r in rentang_waktu) and jam != 12 and not (sesi_asisten.hari == sesi_dosen.hari and jam < sesi_dosen.jam_selesai and (jam + matkul['sks_akademik']) > sesi_dosen.jam_mulai):
                            status = True
                            conflict = False
                            sesi_asisten.jam_mulai = jam
                            sesi_asisten.jam_selesai = jam + matkul['sks_akademik']
                            break
                        else:
                            status = False

                    if not status:
                        excluded_day.append(sesi_asisten.hari)
                        # print('ASISTEN pil ', suggested_hari_asisten)
                        # print('ASISTEN exc ', excluded_day)
                        # print('ASISTEN     ', [d for d in suggested_hari_asisten if d not in excluded_day])
                        # print('      ', available_room_schedule)
                        if all(d in excluded_day for d in suggested_hari_asisten):
                            excluded_day = []
                            excluded_room.append(sesi_asisten.kode_ruangan)
                            ruang_pengganti = rand_ruangan(list_ruangan=ruang_list, data_matkul=matkul, excluded_room=excluded_room)
                            if ruang_pengganti:
                                sesi_asisten.kode_ruangan = ruang_pengganti['kode']
                                sesi_asisten.kapasitas = ruang_pengganti['kapasitas']
                                sesi_asisten.tipe_kelas = ruang_pengganti['tipe_ruangan']
                            else:
                                suggested_hari_asisten = pilihan_hari_asisten
                        else:
                            sesi_asisten.hari = random.choice([d for d in suggested_hari_asisten if d not in excluded_day])

                    attempt += 1
                
                jadwal_ruangan[sesi_asisten.kode_ruangan].append({'hari': sesi_asisten.hari, 'jam_mulai': sesi_asisten.jam_mulai, 'jam_selesai': sesi_asisten.jam_selesai})
    
    # # SWITCH SCHEDULE DOSEN x ASISTEN
    # for sesi_asisten in jadwal:
    #     if sesi_asisten.kode_matkul.endswith("-AS"):
    #         sesi_dosen = next((sd for sd in jadwal if sd.kode_matkul == sesi_asisten.kode_matkul[:-3]), None)
            
    #         tipe_ruang_dosen = next((r['tipe_ruangan'] for r in ruang_list if r['kode'] == sesi_dosen.kode_ruangan), None)
    #         tipe_ruang_asisten = next((r['tipe_ruangan'] for r in ruang_list if r['kode'] == sesi_asisten.kode_ruangan), None)

    #         if tipe_ruang_dosen == tipe_ruang_asisten:
    #             if pilihan_hari_dosen.index(sesi_dosen.hari) >= pilihan_hari_asisten.index(sesi_asisten.hari):
    #                 if sesi_dosen.jam_mulai > sesi_asisten.jam_selesai:
    #                     sesi_dosen.kode_ruangan, sesi_asisten.kode_ruangan = sesi_asisten.kode_ruangan, sesi_dosen.kode_ruangan
    #                     sesi_dosen.hari, sesi_asisten.hari = sesi_asisten.hari, sesi_dosen.hari
    #                     sesi_dosen.jam_mulai, sesi_asisten.jam_mulai = sesi_asisten.jam_mulai, sesi_dosen.jam_mulai
    #                     sesi_dosen.jam_selesai, sesi_asisten.jam_selesai = sesi_asisten.jam_selesai, sesi_dosen.jam_selesai
    #                     # print(f"{'':<3}Switched {sesi_dosen.kode_matkul} <--> {sesi_asisten.kode_matkul}")
        
    return jadwal

In [7]:
def generate_jadwal(matakuliah_list, dosen_list, ruang_list):
    # print(f"{'':<8}{'[ GA ]':<7} Generate Jadwal")
    jadwal = []
    pilihan_hari_dosen = ["SENIN", "SELASA", "RABU", "KAMIS", "JUMAT"]
    pilihan_hari_asisten: list[str] = copy.deepcopy(pilihan_hari_dosen)
    pilihan_hari_asisten.append("SABTU")

    jadwal_dosen = {}    # {nip: [ {hari, jam_mulai, jam_selesai}, ... ]}
    jadwal_ruangan = {}  # {kode_ruangan: [ {hari, jam_mulai, jam_selesai}, ... ]}
    beban_dosen = {} # {nip: beban_sks}

    for dosen in dosen_list:
        jadwal_dosen[dosen['nip']] = []
        beban_dosen[dosen['nip']] = 0
    for ruangan in ruang_list:
        jadwal_ruangan[ruangan['kode']] = []

    max_attempt = 10
    
    for matkul in matakuliah_list:
        if matkul.get('jumlah_kelas'):
            putaran_kelas = int(matkul.get('jumlah_kelas', 0))
        else:
            putaran_kelas = int(matkul['jumlah_mahasiswa'] or 0)

        index_kelas = 1

        dosen_pakar = [
            dosen for dosen in dosen_list
            if (dosen.get("prodi") == matkul['prodi'] or dosen['status'] == "TIDAK_TETAP") and 
                ((
                    (dosen.get('nama') or '') in (matkul.get('dosen_ajar') or []) or 
                    len(set(dosen.get('pakar') or []) & set(matkul.get('bidang') or [])) > 0
                ) if matkul.get('bidang') or matkul.get('dosen_ajar') 
                else True)
        ]

        while putaran_kelas > 0:
            excluded_dosen = []
            ruang_dosen = rand_ruangan(
                list_ruangan=ruang_list, 
                data_matkul=matkul, 
            )
            
            # GENERATE KELAS DOSEN SESUAI DENGAN JUMLAH_DOSEN (TEAM_TEACHING) OR 1
            jumlah_dosen = matkul.get('jumlah_dosen', 1)
            hitung_dosen = 1
            while hitung_dosen <= jumlah_dosen:
                # PEMILIHAN DOSEN
                dosen = rand_dosen_pakar(
                    list_dosen_pakar=dosen_pakar, 
                    dict_beban_sks_dosen=beban_dosen, 
                    excluded_dosen=excluded_dosen
                )

                if hitung_dosen == 1:
                    preferensi_hari = [hari for hari in pilihan_hari_dosen if hari not in dosen['preferensi']['hindari_hari']] if dosen.get('preferensi') and dosen['preferensi'].get('hindari_hari') else pilihan_hari_dosen
                    hari_dosen = random.choice(preferensi_hari)
                    jadwal_dosen_kosong = find_available_schedule(
                        jadwal=jadwal_dosen, 
                        kode=dosen['nip'], 
                        hari=hari_dosen
                    )

                    sukses = False
                    attempt = 0
                    excluded_day = []
                    excluded_room = []
                    while not sukses and max_attempt >= attempt:
                        jadwal_ruangan_kosong = find_available_schedule(
                            jadwal=jadwal_ruangan, 
                            kode=ruang_dosen['kode'], 
                            hari=hari_dosen
                        )

                        available_schedule = list(set(jadwal_ruangan_kosong) & set(jadwal_dosen_kosong))

                        status = False
                        for jam in available_schedule:
                            rentang_waktu = list(range(jam, jam + matkul['sks_akademik'] + 1))
                            if all(r in available_schedule for r in rentang_waktu) and jam != 12:
                                status = True
                                sukses = True
                                jam_mulai_dosen = jam
                                jam_selesai_dosen = jam + matkul['sks_akademik']
                                break
                            else:
                                status = False

                        if not status:
                            excluded_day.append(hari_dosen)
                            # print('DOSEN pil ', pilihan_hari_dosen)
                            # print('DOSEN exc ', excluded_day)
                            # print('DOSEN     ', [d for d in pilihan_hari_dosen if d not in excluded_day])
                            # print('      ', available_schedule)
                            if all(d in excluded_day for d in preferensi_hari):
                                excluded_day = []
                                excluded_room.append(ruang_dosen['kode'])
                                ruang_dosen = rand_ruangan(
                                    list_ruangan=ruang_list, 
                                    data_matkul=matkul, 
                                    excluded_room=excluded_room
                                )
                            else:
                                hari_dosen = random.choice([d for d in preferensi_hari if d not in excluded_day])

                        attempt += 1
                else:
                    conflict = False
                    for sesi_lain in jadwal_dosen[dosen['nip']]:
                        if hari_dosen == sesi_lain['hari']:
                            if jam_mulai_dosen < sesi_lain['jam_selesai'] and jam_selesai_dosen > sesi_lain['jam_mulai']:
                                conflict = True
                                break

                    attempt = 1
                    # Repair Jadwal Dosen Bentrok
                    while conflict and attempt <= max_attempt:
                        excluded_dosen.append(dosen['nip'])
                        if all(dosen["nip"] in excluded_dosen for dosen in dosen_pakar):
                            # print('dosen full')
                            hitung_dosen = jumlah_dosen
                            break
                        else:
                            dosen = rand_dosen_pakar(
                                list_dosen_pakar=dosen_pakar, 
                                dict_beban_sks_dosen=beban_dosen, 
                                excluded_dosen=excluded_dosen
                            )
                        
                        attempt += 1
                
                excluded_dosen.append(dosen["nip"])
                sesi_dosen = JadwalKuliah(
                    kapasitas       = ruang_dosen['kapasitas'],
                    kode_matkul     = f"{matkul['kode']}{angka_ke_huruf(index_kelas)}",
                    kode_dosen      = dosen['nip'],
                    sks_akademik    = matkul['sks_akademik'],
                    kode_ruangan    = ruang_dosen['kode'],
                    hari            = hari_dosen,
                    jam_mulai       = jam_mulai_dosen,
                    jam_selesai     = jam_selesai_dosen,
                    tipe_kelas      = matkul['tipe_kelas'],
                    program_studi   = matkul['prodi']
                )
                jadwal.append(sesi_dosen)

                beban_dosen[dosen['nip']] += matkul['sks_akademik']
                jadwal_dosen[dosen['nip']].append({'hari': hari_dosen, 'jam_mulai': jam_mulai_dosen, 'jam_selesai': jam_selesai_dosen})
                hitung_dosen += 1
            jadwal_ruangan[ruang_dosen['kode']].append({'hari': hari_dosen, 'jam_mulai': jam_mulai_dosen, 'jam_selesai': jam_selesai_dosen})

            # Penentuan kelas asistensi yang tidak terintegrasi dengan kelas dosen
            if matkul.get('asistensi') and not matkul.get('integrated_class'):
                # print(f"{matkul['nama']:<50} praktikum: {matkul.get('asistensi'):<10} integrated: {matkul.get('integrated_class')}")
                # print(f"kapasitas ruangan dosen {ruang_dosen['kapasitas']}; {min(40, max(40, ruang_dosen['kapasitas']))}")
                suggested_hari_asisten = pilihan_hari_asisten[pilihan_hari_dosen.index(hari_dosen):]
                hari_asisten = random.choice(suggested_hari_asisten)

                ruang_asisten = rand_ruangan(
                    list_ruangan=ruang_list, 
                    data_matkul=matkul, 
                    forAsisten=True, 
                    kapasitas_ruangan_dosen=sesi_dosen.kapasitas
                )
                
                sukses = False
                attempt = 0
                excluded_day = []
                excluded_room = []
                while not sukses and max_attempt >= attempt:
                    jadwal_ruangan_kosong = find_available_schedule(
                        jadwal=jadwal_ruangan, 
                        kode=ruang_asisten['kode'], 
                        hari=hari_asisten
                    )

                    status = False
                    for jam in jadwal_ruangan_kosong:
                        rentang_waktu = list(range(jam, jam + matkul['sks_akademik'] + 1))
                        if all(r in jadwal_ruangan_kosong for r in rentang_waktu) and jam != 12 and not (hari_asisten == sesi_dosen.hari and jam < sesi_dosen.jam_selesai and (jam + matkul['sks_akademik']) > sesi_dosen.jam_mulai):
                            status = True
                            sukses = True
                            jam_mulai_asisten = jam
                            jam_selesai_asisten = jam + matkul['sks_akademik']
                            break
                        else:
                            status = False

                    if not status:
                        excluded_day.append(hari_asisten)
                        # print('ASISTEN pil ', suggested_hari_asisten)
                        # print('ASISTEN exc ', excluded_day)
                        # print('ASISTEN     ', [d for d in suggested_hari_asisten if d not in excluded_day])
                        # print('      ', available_room_schedule)
                        if all(d in excluded_day for d in suggested_hari_asisten):
                            excluded_day = []
                            excluded_room.append(ruangan['kode'])
                            ruang_asisten = rand_ruangan(
                                list_ruangan=ruang_list, 
                                data_matkul=matkul, 
                                excluded_room=excluded_room, 
                                forAsisten=True, 
                                kapasitas_ruangan_dosen=sesi_dosen.kapasitas
                            )
                        else:
                            hari_asisten = random.choice([d for d in suggested_hari_asisten if d not in excluded_day])

                    attempt += 1
                
                sesi_asisten = JadwalKuliah(
                    kapasitas    = ruang_asisten['kapasitas'],
                    kode_matkul  = f"{matkul['kode']}{angka_ke_huruf(index_kelas)}-AS",
                    kode_dosen   = "AS",
                    sks_akademik = matkul['sks_akademik'],
                    kode_ruangan = ruang_asisten['kode'],
                    hari         = hari_asisten,
                    jam_mulai    = jam_mulai_asisten,
                    jam_selesai  = jam_selesai_asisten,
                    tipe_kelas   = matkul['tipe_kelas_asistensi'],
                    program_studi= matkul['prodi']
                )
                jadwal.append(sesi_asisten)
                jadwal_ruangan[ruang_asisten['kode']].append({'hari': hari_asisten, 'jam_mulai': jam_mulai_asisten, 'jam_selesai': jam_selesai_asisten})
            
            putaran_kelas -= (1 if matkul.get('jumlah_kelas') else ruang_dosen['kapasitas'])
            index_kelas += 1

    return jadwal

In [8]:
def hitung_fitness(jadwal, matakuliah_list, dosen_list, ruang_list, detail=False):
    penalti = 0
    jadwal_dosen = {}
    jadwal_ruangan = {}
    beban_dosen = {}

    for dosen in dosen_list:
        if dosen['nip'] not in jadwal_dosen: jadwal_dosen[dosen['nip']] = []
        if dosen['nip'] not in beban_dosen: beban_dosen[dosen['nip']] = 0

    for ruangan in ruang_list:
        if ruangan['kode'] not in jadwal_ruangan: jadwal_ruangan[ruangan['kode']] = []

    # COUNTER
    hitung_ruangan_bentrok = 0
    hitung_dosen_bentrok = 0
    hitung_asdos_nabrak_dosen = 0
    hitung_kelas_dosen_missing = 0
    hitung_kelas_asisten_missing = 0
    hitung_diluar_jam_kerja = 0
    hitung_solo_team = {}
    pelanggaran_preferensi = {}
    mata_kuliah_minus = {}

    detail_jadwal_matkul = {}
    detail_team = {}

    # TO BE CHECKED:
    # (15)  Jadwal Ruangan Bertabrakan                                  >> ruangan_bentrok           (DONE)
    # (15)  Jadwal Dosen Bertabrakan                                    >> dosen_bentrok             (DONE)
    # (15)  Jadwal Dosen dan Asisten Berjalan Bersamaan                 >> asdos_nabrak_dosen        (DONE)
    # (15)  Kelas Dosen atau Asisten Hilang atau Tidak Lengkap          >> kelas_gaib                (DONE)
    # (10)  Beban SKS Dosen melebihi 12 sks                             >> dosen_overdosis           (DONE)
    # (10)  Matkul berlangsung sebelum pukul 7 atau sesudah pukul 19    >> diluar_jam_kerja          (DONE)
    # (10)  Cek Total Kelas Bisa Cangkup Semua Mahasiswa                >> kapasitas_kelas_terbatas  (DONE)
    # (5)   Tidak Sesuai dengan permintaan / request dosen              >> melanggar_preferensi      (DONE)

    seen_course = set()
    for sesi in jadwal:
        kode_matkul = sesi.kode_matkul[:-1] if sesi.kode_dosen != "AS" else sesi.kode_matkul[:-4]
        info_matkul = next((m for m in matakuliah_list if m['kode'] == kode_matkul), None)
        info_dosen = next((d for d in dosen_list if d['nip'] == sesi.kode_dosen), None)
        
        if kode_matkul not in detail_jadwal_matkul: detail_jadwal_matkul[kode_matkul] = {'jumlah_kelas': 0, 'kapasitas_total': 0}
        if sesi.kode_matkul not in detail_team and sesi.kode_dosen != "AS": detail_team[sesi.kode_matkul] = []

        # CEK SUPAYA BENTROK RUANGAN CUMA DICEK 1x UNTUK KODE MATKUL YANG SAMA
        if sesi.kode_matkul not in seen_course:
            seen_course.add(sesi.kode_matkul)
            
            # CEK BENTROK JADWAL RUANGAN
            for sesi_lain in jadwal_ruangan[sesi.kode_ruangan]:
                if sesi.hari == sesi_lain['hari']:
                    if sesi.jam_mulai < sesi_lain['jam_selesai'] and sesi.jam_selesai > sesi_lain['jam_mulai']:
                        penalti += BOBOT_PENALTI['ruangan_bentrok']
                        hitung_ruangan_bentrok += 1
            jadwal_ruangan[sesi.kode_ruangan].append({'hari': sesi.hari, 'jam_mulai': sesi.jam_mulai, 'jam_selesai': sesi.jam_selesai})

            # CEK JAM MULAI DAN JAM SELESAI MASIH DI JAM KERJA ATAU TIDAK
            if sesi.jam_mulai < 7 or sesi.jam_selesai > 19:
                penalti += BOBOT_PENALTI['diluar_jam_kerja']
                hitung_diluar_jam_kerja += 1

            if sesi.kode_dosen != "AS":
                # HITUNG TOTAL KAPASITAS
                detail_jadwal_matkul[kode_matkul]['jumlah_kelas'] += 1
                detail_jadwal_matkul[kode_matkul]['kapasitas_total'] += sesi.kapasitas

                # CEK EKSISTENSI KELAS ASISTEN
                if info_matkul.get('asistensi') and not info_matkul.get('integrated_class'):
                    sesi_asisten = next((sa for sa in jadwal if sa.kode_matkul == sesi.kode_matkul+"-AS"), None)
                    if sesi_asisten:
                        # CEK BENTROK JADWAL DOSEN X ASISTEN NGGA BENER INI ANJENG. CEK ULANG SU
                        if sesi.hari == sesi_asisten.hari:
                            if sesi.jam_mulai < sesi_asisten.jam_selesai and sesi.jam_selesai > sesi_asisten.jam_mulai:
                                penalti += BOBOT_PENALTI['asdos_nabrak_dosen']
                                hitung_asdos_nabrak_dosen += 1
                    else:
                        penalti += BOBOT_PENALTI['kelas_gaib']
                        hitung_kelas_asisten_missing += 1

        if info_matkul.get('team_teaching') and sesi.kode_dosen != "AS":
            if info_dosen['nip'] in detail_team[sesi.kode_matkul]:
                if info_dosen['nip'] not in hitung_solo_team: hitung_solo_team[info_dosen['nip']] = 0
                hitung_solo_team[info_dosen['nip']] += 1
                penalti += BOBOT_PENALTI['solo_team']
            detail_team[sesi.kode_matkul].append(info_dosen['nip'])


        if sesi.kode_dosen != "AS":
            # CEK BENTROK JADWAL DOSEN
            for sesi_lain in jadwal_dosen[sesi.kode_dosen]:
                if sesi.hari == sesi_lain['hari']:
                    if sesi.jam_mulai < sesi_lain['jam_selesai'] and sesi.jam_selesai > sesi_lain['jam_mulai']:
                        penalti += BOBOT_PENALTI['dosen_bentrok']
                        hitung_dosen_bentrok += 1
            jadwal_dosen[sesi.kode_dosen].append({'hari': sesi.hari, 'jam_mulai': sesi.jam_mulai, 'jam_selesai': sesi.jam_selesai})

            # CEK PELANGGARAN BEBAN SKS DOSEN
            beban_dosen[sesi.kode_dosen] += sesi.sks_akademik
            if beban_dosen[sesi.kode_dosen] > 12:
                penalti += BOBOT_PENALTI['dosen_overdosis']

            # CEK PELANGGARAN PREFERENSI DOSEN
            preferensi_dosen = info_dosen.get("preferensi", None)
            if preferensi_dosen:
                hindari_hari = preferensi_dosen.get("hindari_hari", None)
                hindari_jam = preferensi_dosen.get("hindari_jam", None)
                if (hindari_hari and sesi.hari in hindari_hari) or (hindari_jam and any(jam in hindari_jam for jam in range(sesi.jam_mulai, sesi.jam_selesai + 1))):
                    if info_dosen['nip'] not in pelanggaran_preferensi: pelanggaran_preferensi[info_dosen['nip']] = []
                    pelanggaran_preferensi[info_dosen['nip']].append(sesi.hari)
                    penalti += BOBOT_PENALTI['melanggar_preferensi']
        else:
            sesi_dosen = next((sd for sd in jadwal if sd.kode_matkul == sesi.kode_matkul[:-3]), None)
            # CEK EKSISTENSI KELAS DOSEN
            if not sesi_dosen:
                penalti += BOBOT_PENALTI['kelas_gaib']
                hitung_kelas_dosen_missing += 1
                
    # CEK KEKURANGAN KAPASITAS TIAP MATKUL
    for kode_matkul, data in detail_jadwal_matkul.items():
        matkul_detail = next((matkul for matkul in matakuliah_list if matkul['kode'] == kode_matkul), None)
        if matkul_detail:
            if matkul_detail.get('jumlah_kelas') and data['jumlah_kelas'] < matkul_detail['jumlah_kelas']:
                penalti += BOBOT_PENALTI['kapasitas_kelas_terbatas']
            elif not matkul_detail.get('jumlah_kelas') and data['kapasitas_total'] < matkul_detail['jumlah_mahasiswa']:
                kekurangan_kapasitas = matkul_detail['jumlah_mahasiswa'] - data['kapasitas_total']
                penalti += (BOBOT_PENALTI['kapasitas_kelas_terbatas'] * (kekurangan_kapasitas/10))
                mata_kuliah_minus[kode_matkul] = kekurangan_kapasitas

    if detail:
        if beban_dosen: print(f"{'':<10}{'beban dosen':<40}: {beban_dosen}")
        if hitung_dosen_bentrok: print(f"{'':<10}{'Bentrok Dosen':<40} : {hitung_dosen_bentrok}")
        if hitung_ruangan_bentrok: print(f"{'':<10}{'Bentrok Ruangan':<40} : {hitung_ruangan_bentrok}")
        if hitung_asdos_nabrak_dosen: print(f"{'':<10}{'Bentrok Dosen-Asdos':<40} : {hitung_asdos_nabrak_dosen}")
        if hitung_diluar_jam_kerja: print(f"{'':<10}{'Kelas Diluar Jam Kerja':<40} : {hitung_diluar_jam_kerja}")
        if hitung_kelas_dosen_missing: print(f"{'':<10}{'Kelas Dosen Missing':<40} : {hitung_kelas_dosen_missing}")
        if hitung_kelas_asisten_missing: print(f"{'':<10}{'Kelas Asisten Missing':<40} : {hitung_kelas_asisten_missing}")
        if mata_kuliah_minus: print(f"{'':<10}{'Kapasitas kelas kurang x':<40} : {mata_kuliah_minus}")
        hitung_solo_team = {k: v for k, v in hitung_solo_team.items() if v}
        pelanggaran_preferensi = {k: v for k, v in pelanggaran_preferensi.items() if v}
        print(f"{'solo team':<50}: {hitung_solo_team}")
        print(f"{'pelanggaran preferensi':<50}: {pelanggaran_preferensi}")

    # print(f"{'final fitness':<50} : {max(0, 1000 - penalti)}")
    # return max(0, 1000 - penalti)
    return 1000-penalti

In [9]:
def generate_populasi(matakuliah_list, dosen_list, ruang_list, ukuran_populasi):
    print(f"{'':<8}{'[ GA ]':<7} Generate Generasi Pertama")
    return [generate_jadwal(matakuliah_list, dosen_list, ruang_list) for _ in range(ukuran_populasi)]

def roulette_selection(populasi, fitness_scores):
    """
    Memilih satu individu dari populasi menggunakan metode seleksi berdasarkan fitness.

    Individu dipilih secara acak menggunakan nilai acak antara 0 hingga total fitness 
    (populasi), di mana individu dengan fitness lebih tinggi memiliki peluang lebih besar terpilih.

    Args:
        populasi (list): Daftar individu (jadwal) yang tersedia.
        fitness_scores (list): Daftar nilai fitness yang sesuai dengan setiap individu dalam populasi.

    Returns:
        object: Individu yang terpilih dari populasi.
    """
    
    total_fitness = sum(fitness_scores)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individu, score in zip(populasi, fitness_scores):
        current += score
        if current > pick:
            return individu
    return random.choice(populasi)  # fallback, jika tidak ketemu

def crossover(parent1: object, parent2: object):
    """
    Melakukan crossover antara parent1 dan parent2 berdasarkan blok kode_matkul (tanpa suffix paralel).

    Titik acak diambil dari parent1, lalu blok matkul utamanya diidentifikasi
    (misal TC502 dari TC502A/TC502C/TC502-AS). Crossover dilakukan dari titik
    awal blok tersebut pada kedua parent.

    Args:
        parent1 (list): Individu pertama berupa list sesi.
        parent2 (list): Individu kedua berupa list sesi.

    Returns:
        tuple: Dua individu hasil crossover (child1, child2).
    """
    
    titik = random.randint(1, len(parent1)-1)
    sesi_target = parent1[titik]
    kode_matkul = sesi_target.kode_matkul
    is_asisten = sesi_target.kode_dosen == "AS"
    matkul_code_base = kode_matkul[:-4] if is_asisten else kode_matkul[:-1]

    titik1 = next((index for index, sesi in enumerate(parent1) if sesi.kode_matkul == matkul_code_base + 'A'), 0)
    titik2 = next((index for index, sesi in enumerate(parent2) if sesi.kode_matkul == matkul_code_base + 'A'), 0)

    child1 = parent1[:titik1] + parent2[titik2:]
    child2 = parent2[:titik2] + parent1[titik1:]
    return child1, child2

def mutasi(individu, dosen_list, matakuliah_list, ruang_list, peluang_mutasi=0.1):
    """
    Melakukan mutasi pada individu (jadwal) secara acak berdasarkan peluang yang ditentukan.

    Setiap sesi dalam individu memiliki kemungkinan untuk dimodifikasi secara acak 
    pada atribut `hari`, `jam_mulai` + `jam_selesai`, atau `kode_ruangan`.

    Args:
        individu (list): Daftar sesi jadwal dalam satu individu.
        dosen_list (list): Daftar dosen yang tersedia.
        ruang_list (list): Daftar ruangan yang tersedia, masing-masing berupa dict dengan key 'kode'.
        peluang_mutasi (float, optional): Peluang untuk setiap sesi dimutasi. Default 0.1.

    Returns:
        list: Individu yang telah mengalami mutasi (bisa sama atau berbeda).
    """
    
    pilihan_hari_dosen = ["SENIN", "SELASA", "RABU", "KAMIS", "JUMAT"]
    pilihan_hari_asisten = copy.deepcopy(pilihan_hari_dosen)
    pilihan_hari_asisten.append("SABTU")
    pilihan_waktu = list(range(7, 20))

    for sesi in individu:
        if random.random() < peluang_mutasi:
            # Randomly mutate hari, jam, atau ruangan
            attr = random.choice(['hari', 'jam', 'ruang'])

            if attr == 'hari':
                sesi.hari = random.choice(pilihan_hari_dosen) if sesi.kode_dosen != "AS" else random.choice(pilihan_hari_asisten)
            elif attr == 'jam':
                jam_mulai = random.choice(pilihan_waktu)
                sesi.jam_mulai = (
                    jam_mulai 
                    if jam_mulai + sesi.sks_akademik <= max(pilihan_waktu) 
                    else max(pilihan_waktu) - sesi.sks_akademik)
                sesi.jam_selesai = sesi.jam_mulai + sesi.sks_akademik
            elif attr == 'ruang':
                kode_matkul = sesi.kode_matkul[:-1] if sesi.kode_dosen != "AS" else sesi.kode_matkul[:-4]
                matkul = next((matkul for matkul in matakuliah_list if matkul['kode'] == kode_matkul), None)
                if sesi.kode_dosen != "AS":
                    calon_ruang_pengganti = [
                        r for r in ruang_list if 
                            (matkul['prodi'] in r['plot'] or 'GENERAL' in r['plot']) and
                            r['kapasitas'] >= 35 and
                            (
                                r['tipe_ruangan'] == sesi.tipe_kelas or 
                                (r['tipe_ruangan'] == "RAPAT" if not matkul.get('asistensi') else False)
                            )
                    ]
                else:
                    calon_ruang_pengganti = [
                        r for r in ruang_list if 
                            (matkul['prodi'] in r['plot'] or 'GENERAL' in r['plot']) and
                            r['kapasitas'] >= sesi.kapasitas and
                            r['tipe_ruangan'] == matkul['tipe_kelas_asistensi']
                    ]
                bobot_calon_ruang_pengganti = [
                    (len(set(r['plot']) & set(matkul.get('bidang') or []))*10 or 1)
                    for r in calon_ruang_pengganti
                ]
                ruang_pengganti = random.choices(
                    population=calon_ruang_pengganti, 
                    weights=bobot_calon_ruang_pengganti, 
                    k=1)[0]
                sesi.kode_ruangan = ruang_pengganti['kode']
    return individu

def genetic_algorithm(matakuliah_list, dosen_list, ruang_list, ukuran_populasi=75, jumlah_generasi=100, peluang_mutasi=0.1, proporsi_elite=0.05):
    populasi = generate_populasi(matakuliah_list, dosen_list, ruang_list, ukuran_populasi)
    populasi = [repair_jadwal(j, matakuliah_list, dosen_list, ruang_list) for j in populasi]
    
    best_fitness_global = float('-inf')
    best_individual_global = None

    # jumlah_elite = max(1, int(ukuran_populasi * proporsi_elite))

    for gen in range(jumlah_generasi):
        fitness_scores = [hitung_fitness(individu, matakuliah_list, dosen_list, ruang_list) for individu in populasi]
        next_gen = []

        # # Simpan individu terbaik
        # individu_elite = [individu for _, individu in sorted(zip(fitness_scores, populasi), key=lambda x: x[0], reverse=True)][:jumlah_elite]
        # next_gen = individu_elite.copy()

        # Generate anak baru
        while len(next_gen) < ukuran_populasi:
            parent1 = roulette_selection(populasi, fitness_scores)
            parent2 = roulette_selection(populasi, fitness_scores)

            child1, child2 = crossover(parent1, parent2)

            child1 = mutasi(child1, dosen_list, matakuliah_list, ruang_list, peluang_mutasi)
            child2 = mutasi(child2, dosen_list, matakuliah_list, ruang_list, peluang_mutasi)

            child1 = repair_jadwal(child1, matakuliah_list, dosen_list, ruang_list)
            child2 = repair_jadwal(child2, matakuliah_list, dosen_list, ruang_list)

            next_gen.append(child1)
            if len(next_gen) < ukuran_populasi:
                next_gen.append(child2)
            
        populasi = next_gen

        fitness_scores = [hitung_fitness(individu, matakuliah_list, dosen_list, ruang_list) for individu in populasi]
        gen_best_fitness = max(fitness_scores)
        gen_best_individual = populasi[fitness_scores.index(gen_best_fitness)]
        
        if gen_best_fitness >= best_fitness_global:
            best_fitness_global = gen_best_fitness
            best_individual_global = copy.deepcopy(gen_best_individual)

        # best_fitness_global = gen_best_fitness
        # best_individual_global = copy.deepcopy(gen_best_individual)

        avg = sum(fitness_scores) / len(fitness_scores)
        print(f"[Gen {gen}] [({len(populasi)} population)] AVG : {round(avg, 2):<10} BEST ALLTIME: {best_fitness_global}")
        print(f"{f'[Gen {gen}]':<10}Min: {min(fitness_scores):<5}Max: {max(fitness_scores):<5}Best Gen: {hitung_fitness(gen_best_individual, matakuliah_list, dosen_list, ruang_list, True)}")
        print(f"gen best fitness {gen_best_fitness}")
        # hitung_fitness(gen_best_individual, matakuliah_list, dosen_list, ruang_list, True)
        print(f"{'':<5}Missing: {find_missing_course(best_individual_global, matakuliah_list)}\n") if find_missing_course(best_individual_global, matakuliah_list) else print("\n")

    print(f"GLOBAL BEST FITNESS {best_fitness_global}, LAST GEN WORST INDIVIDUAL {min(fitness_scores)}")
    # CEK BEST GLOBAL
    hitung_fitness(best_individual_global, matakuliah_list, dosen_list, ruang_list, True)
    return best_individual_global
    # # CEK INDIVIDU TERBURUK
    # hitung_fitness(populasi[fitness_scores.index(min(fitness_scores))], matakuliah_list, dosen_list, ruang_list, True)
    # return populasi[fitness_scores.index(min(fitness_scores))]

In [ ]:
# TESTING REORGANIZE MATKUL
reorganize_matkul = []
seen_matkul = []
print(next((dosen.get('matkul_ajar') for dosen in dosen_list if dosen["nama"] == "PRATYAKSA OCSA NUGRAHA SAIAN, S.KOM., M.T"), None))

# SORTING BERDASARKAN FIELD TERBANYAK YANG BISA DIAJAR
for dosen in sorted(dosen_list, key=lambda dosen: (len(dosen.get("matkul_ajar", [])), len(dosen.get('pakar', []))), reverse=True):
    for nama_matkul in dosen.get("matkul_ajar", []):
        data_matkul = next((matkul for matkul in matakuliah_list if matkul["nama"] == nama_matkul), None)
        if data_matkul and data_matkul['kode'] not in seen_matkul:
            reorganize_matkul.append(data_matkul)
            seen_matkul.append(data_matkul['kode'])

In [15]:
jadwal = generate_jadwal(matakuliah_list, dosen_list, ruang_list)
print(hitung_fitness(jadwal, matakuliah_list, dosen_list, ruang_list, True))
# jadwal = repair_jadwal(jadwal, matakuliah_list, dosen_list, ruang_list)
# print(hitung_fitness(jadwal, matakuliah_list, dosen_list, ruang_list, True))

# jadwal = generate_populasi(reorganize_matkul, dosen_list, ruang_list, 75)
# fitness = [hitung_fitness(sesi, reorganize_matkul, dosen_list, ruang_list, False) for sesi in jadwal]
# print(min(fitness), max(fitness))
# jadwal = jadwal[fitness.index(max(fitness))]
# print(hitung_fitness(jadwal, matakuliah_list, dosen_list, ruang_list, True))

# tampilkan_jadwal(jadwal)
export_jadwal_to_excel(jadwal)
export_ruangan_to_excel(jadwal)

          beban dosen                             : {'67564': 2, '67909': 6, '67530': 9, '67003': 3, '67920': 9, '67555': 9, '67513': 9, '67523': 12, '67515': 9, '67002': 9, '67938': 12, '68975': 15, '67565': 14, '67506': 12, '67545': 9, '67554': 3, '67501': 9, '67507': 9, '67977': 3, '67532': 9, '56907': 9, '67566': 9, '55905': 9, '67941': 5, '67578': 6, '67570': 9, '67922': 9, '67810': 9, '67916': 4, '67927': 0, '67574': 0, '68972': 9, '67947': 6, '67975': 6, '68973': 4, '67967': 9, '11955': 2, '00000': 0, '00001': 0, '67925': 0, '67524': 6, '67512': 9, '67955': 6, '67005': 12, '67001': 0, '67517': 0, '67004': 3, '67508': 0, '55909': 12, '67503': 6, '67942': 0, '67560': 9, '67511': 0}
          Bentrok Dosen                            : 1
solo team                                         : {}
pelanggaran preferensi                            : {}
965
✅ File 'jadwal_kuliah.xlsx' berhasil dibuat.
Excel disimpan di: /home/kevinprasetio/course_scheduling/dao
✅ File 'jadwal_ruangan.xlsx' 

/home/kevinprasetio/course_scheduling/dao/jadwal_ruangan.xlsx

In [ ]:
# generate_populasi(matakuliah_list, dosen_list, ruang_list, 100)
# best_schedule = genetic_algorithm(matakuliah_list, dosen_list, ruang_list)
best_schedule = genetic_algorithm(
    matakuliah_list, dosen_list, ruang_list, 
    ukuran_populasi=75, jumlah_generasi=100, peluang_mutasi=0.05
)
# print('\n')
# tampilkan_jadwal(best_schedule)
export_jadwal_to_excel(best_schedule)
# tampilkan_jadwal(best_schedule)